## Identifying and resolving conflicts in a batch of VI files

ELGs - part 5 is sets 21 to 25 

There are 750 visual inspections of a total of 250 unique objects
3.0 inspections per object.

81 conflicts

In [1]:
save_filename='truth_table_ELGs_sets21_25.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(21,26):
    pattern = "desi*_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

vi_files

desi*_21_*.csv
desi*_22_*.csv
desi*_23_*.csv
desi*_24_*.csv
desi*_25_*.csv


['desi-vi_SV0_ELG_tile67230_night20200315_21_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_21_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_21_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_22_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_22_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_22_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_23_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_23_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_23_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_24_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_24_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_24_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_24_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_25_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_25_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_25_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_25_CGS.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_21_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_21_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_22_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_22_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_22_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_23_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_23_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_23_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_24_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_24_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_24_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_24_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_25_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_25_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_25_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_25_CGS.csv
There are 850 visual inspections of a total of 250 unique objects
That gives 3.4 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
850


### Adding a bunch of useful columns

In [11]:
vi = VI.prep_new_columns(vi)

In [12]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185766111903950,-1,0.33.0.dev3890,0,QSO,1.1209,TWL,0,C,--,...,1.758992,1.1209,QSO,1.000000,3,0.447782,1.1209,Very low S/N|Noisy spectra|Strong Lyman alpha,3,none
1,35185766111904437,-1,0.33.0.dev3890,0,GALAXY,0.8573,TWL,4,--,--,...,5122.273197,0.8573,GALAXY,4.000000,0,0.000000,0.8573,--|--| MgII in absorption,3,none
2,35185766116098303,-1,0.33.0.dev3890,0,GALAXY,0.9031,TWL,4,--,--,...,248.966164,0.9031,GALAXY,3.666667,1,0.000000,0.9031,--|OII doublet is resolved as one wide line|--,3,none
3,35185766116098370,-1,0.33.0.dev3890,0,GALAXY,0.4881,TWL,2,--,--,...,400.307992,0.4881,GALAXY,2.666667,2,0.000000,0.4881,Model fits to [OIII] lines but no [OII] lines...,3,none
4,35185766116098608,-1,0.33.0.dev3890,0,GALAXY,0.7262,TWL,4,--,--,...,330.734332,0.7262,GALAXY,4.000000,0,0.000000,0.7262,--|--|--,3,none
5,35185766116098852,-1,0.33.0.dev3890,0,GALAXY,0.3585,TWL,0,--,--,...,21.962877,0.3585,GALAXY,0.666667,2,0.631358,0.3585,Model fits negative flux / too low S/N| Noisy...,3,none
6,35185766116098878,-1,0.33.0.dev3890,0,GALAXY,1.1977,TWL,3,--,--,...,20.582957,1.1977,GALAXY,2.333333,2,0.000000,1.1977,--|Noisy spectra OII doublet is resolved as on...,3,none
7,35185766116098895,-1,0.33.0.dev3890,0,GALAXY,1.6939,TWL,0,--,--,...,5.601308,1.6939,GALAXY,0.666667,2,0.080846,1.6939,Too low S/N| Noisy spectra| Weak OII,3,none
8,35185766116099530,-1,0.33.0.dev3890,0,GALAXY,1.2044,TWL,2,--,--,...,196.107786,1.2044,GALAXY,2.000000,2,0.000000,1.2044,Strange [OII] profile| Noisy spectra OII doubl...,3,none
9,35185766116100016,-1,0.33.0.dev3890,0,GALAXY,0.0093,TWL,0,RC,--,...,169.499487,0.0093,GALAXY,0.333333,1,0.000000,0.0093,Model fits negative flux| Noisy spectra| No c...,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [13]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [14]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185766111903950 35185766111904318 35185766111904889 35185766111905013
 35185766111905542 35185766111905627 35185766111906265 35185766111906679
 35185766111907627 35185766111908072 35185766111908276 35185766111909116
 35185766116098370 35185766116098878 35185766116099530 35185766116100334
 35185766116101295 35185766116101682 35185766116102188 35185766116102579
 35185766116102675 35185766116102727 35185766116103427 35185766116103536
 35185766120292507 35185766120294735 35185766120295131 35185766120296240
 35185766120296280 35185766120297337 35185766120297470 35185766124486990
 35185766124487574 35185766124488456 35185766124488648 35185766124490426
 35185766124490531 35185766124490928 35185766124490981 35185766124491408
 35185766124491740 35185766128685429 35185766132877398 35185766137071522
 35185766137071782 35185766137073679 35185766137073699 35185766141265461
 35185766141266056 35185766141266205 35185766141267125 35185766141267656
 35185766141268985 35

In [15]:
unique_targets[1]

35185766111904318

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [16]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [17]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [20]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
0,35185766111903950,QSO,1.1209,TWL,0,C,--,--,Very low S/N,1.1209,QSO,1.0,3,1.1209,0.447782,Very low S/N|Noisy spectra|Strong Lyman alpha,3,none
50,35185766111903950,QSO,1.1209,PNT,0,RCS,--,--,Noisy spectra,1.1209,QSO,1.0,3,1.1209,0.447782,Very low S/N|Noisy spectra|Strong Lyman alpha,3,none
100,35185766111903950,QSO,1.1209,ACE,3,--,2.0706,GALAXY,Strong Lyman alpha,2.0706,GALAXY,1.0,3,1.1209,0.447782,Very low S/N|Noisy spectra|Strong Lyman alpha,3,none


In [21]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1209
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. one inspector suggests unlikely high z galaxy due to possible Lyman alpha. low SNR. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185766111903950,-1,0.33.0.dev3890,0,QSO,1.1209,TWL,1,C,--,...,1.758992,1.1209,QSO,1.0,3,0.447782,1.1209,Very low S/N|Noisy spectra|Strong Lyman alpha,3,disagreement in flag. one inspector suggests u...
50,35185766111903950,-1,0.33.0.dev3890,0,QSO,1.1209,PNT,1,RCS,--,...,1.758992,1.1209,QSO,1.0,3,0.447782,1.1209,Very low S/N|Noisy spectra|Strong Lyman alpha,3,disagreement in flag. one inspector suggests u...
100,35185766111903950,-1,0.33.0.dev3890,0,QSO,1.1209,ACE,1,--,2.0706,...,1.758992,1.1209,QSO,1.0,3,0.447782,1.1209,Very low S/N|Noisy spectra|Strong Lyman alpha,3,disagreement in flag. one inspector suggests u...


conflict ID =  0


In [24]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
18,35185766111904318,GALAXY,0.6678,TWL,0,R,--,--,Model fits on sky residuals,0.6678,GALAXY,1.0,3,0.6678,0.343207,Model fits on sky residuals| Noisy spectra| R...,3,none
68,35185766111904318,GALAXY,0.6678,PNT,0,--,--,--,Noisy spectra,0.6678,GALAXY,1.0,3,0.6678,0.343207,Model fits on sky residuals| Noisy spectra| R...,3,none
118,35185766111904318,GALAXY,0.6678,ACE,3,--,1.2402,GALAXY,Redrock confuses OII for OIII,1.2402,GALAXY,1.0,3,0.6678,0.343207,Model fits on sky residuals| Noisy spectra| R...,3,none


In [25]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6678
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. one inspector suggests z based on OII but in skylines. take RR answer with flag 1. poor sky subtraction. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
18,35185766111904318,-1,0.33.0.dev3890,0,GALAXY,0.6678,TWL,1,R,--,...,7.086598,0.6678,GALAXY,1.0,3,0.343207,0.6678,Model fits on sky residuals| Noisy spectra| R...,3,disagreement in flag and z. one inspector sugg...
68,35185766111904318,-1,0.33.0.dev3890,0,GALAXY,0.6678,PNT,1,--,--,...,7.086598,0.6678,GALAXY,1.0,3,0.343207,0.6678,Model fits on sky residuals| Noisy spectra| R...,3,disagreement in flag and z. one inspector sugg...
118,35185766111904318,-1,0.33.0.dev3890,0,GALAXY,0.6678,ACE,1,--,1.2402,...,7.086598,0.6678,GALAXY,1.0,3,0.343207,0.6678,Model fits on sky residuals| Noisy spectra| R...,3,disagreement in flag and z. one inspector sugg...


conflict ID =  1


In [28]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185766111904889,GALAXY,0.5357,TWL,1,--,--,--,Emission line feature at 7680A. Not sure what...,0.5357,GALAXY,2.0,3,0.5357,0.343101,Emission line feature at 7680A. Not sure what...,3,none
70,35185766111904889,GALAXY,0.5357,PNT,1,--,0.5357,GALAXY,Noisy spectra OIII lines are resolved,0.5357,GALAXY,2.0,3,0.5357,0.343101,Emission line feature at 7680A. Not sure what...,3,none
120,35185766111904889,GALAXY,0.5357,ACE,4,--,1.0626,GALAXY,Redrock confuses OII for OIII,1.0626,GALAXY,2.0,3,0.5357,0.343101,Emission line feature at 7680A. Not sure what...,3,none


In [29]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0626
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 1/3 inspectors suggests z on broadened but unresolved line. take with flag 2. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
20,35185766111904889,-1,0.33.0.dev3890,0,GALAXY,0.5357,TWL,2,--,--,...,12.508221,1.0626,GALAXY,2.0,3,0.343101,0.5357,Emission line feature at 7680A. Not sure what...,3,disagreement in flag and z. 1/3 inspectors sug...
70,35185766111904889,-1,0.33.0.dev3890,0,GALAXY,0.5357,PNT,2,--,0.5357,...,12.508221,1.0626,GALAXY,2.0,3,0.343101,0.5357,Emission line feature at 7680A. Not sure what...,3,disagreement in flag and z. 1/3 inspectors sug...
120,35185766111904889,-1,0.33.0.dev3890,0,GALAXY,0.5357,ACE,2,--,1.0626,...,12.508221,1.0626,GALAXY,2.0,3,0.343101,0.5357,Emission line feature at 7680A. Not sure what...,3,disagreement in flag and z. 1/3 inspectors sug...


conflict ID =  2


In [32]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
21,35185766111905013,GALAXY,1.6649,TWL,3,R,1.2787,--,[OII] at z~1.2787,1.2787,GALAXY,2.0,3,1.6649,0.169483,[OII] at z~1.2787| Noisy spectra|--,3,none
71,35185766111905013,GALAXY,1.6649,PNT,0,--,--,--,Noisy spectra,1.6649,GALAXY,2.0,3,1.6649,0.169483,[OII] at z~1.2787| Noisy spectra|--,3,none
121,35185766111905013,GALAXY,1.6649,ACE,3,R,1.2789,GALAXY,--,1.2789,GALAXY,2.0,3,1.6649,0.169483,[OII] at z~1.2787| Noisy spectra|--,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2789
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest z based on missed OII. take with flag 3. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
21,35185766111905013,-1,0.33.0.dev3890,0,GALAXY,1.6649,TWL,3,R,1.2787,...,8.657816,1.2789,GALAXY,2.0,3,0.169483,1.6649,[OII] at z~1.2787| Noisy spectra|--,3,2/3 inspectors suggest z based on missed OII. ...
71,35185766111905013,-1,0.33.0.dev3890,0,GALAXY,1.6649,PNT,3,--,--,...,8.657816,1.2789,GALAXY,2.0,3,0.169483,1.6649,[OII] at z~1.2787| Noisy spectra|--,3,2/3 inspectors suggest z based on missed OII. ...
121,35185766111905013,-1,0.33.0.dev3890,0,GALAXY,1.6649,ACE,3,R,1.2789,...,8.657816,1.2789,GALAXY,2.0,3,0.169483,1.6649,[OII] at z~1.2787| Noisy spectra|--,3,2/3 inspectors suggest z based on missed OII. ...


conflict ID =  3


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
24,35185766111905542,QSO,1.3548,TWL,2,S,--,--,Bad calibration feature around 5800 - 6800. M...,1.3548,QSO,1.333333,4,1.354867,0.000085,Bad calibration feature around 5800 - 6800. M...,3,none
74,35185766111905542,QSO,1.3548,PNT,-1,RCS,--,--,Noisy spectra big jumps gaps change of spectra,1.3548,QSO,1.333333,4,1.354867,0.000085,Bad calibration feature around 5800 - 6800. M...,3,none
124,35185766111905542,QSO,1.3548,ACE,3,RCS,1.3550,GALAXY,Continuum offset 5800-6700A. Based on OII only,1.3550,GALAXY,1.333333,4,1.354867,0.000085,Bad calibration feature around 5800 - 6800. M...,3,none


In [37]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. z based on OII on skylines. not secure. poor sky calibration. calibration issues. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
24,35185766111905542,-1,0.33.0.dev3890,0,QSO,1.3548,TWL,2,S,--,...,115.653612,1.3548,GALAXY,1.333333,4,0.000085,1.354867,Bad calibration feature around 5800 - 6800. M...,3,disagreement in flag and spectype. z based on ...
74,35185766111905542,-1,0.33.0.dev3890,0,QSO,1.3548,PNT,2,RCS,--,...,115.653612,1.3548,GALAXY,1.333333,4,0.000085,1.354867,Bad calibration feature around 5800 - 6800. M...,3,disagreement in flag and spectype. z based on ...
124,35185766111905542,-1,0.33.0.dev3890,0,QSO,1.3548,ACE,2,RCS,1.3550,...,115.653612,1.3550,GALAXY,1.333333,4,0.000085,1.354867,Bad calibration feature around 5800 - 6800. M...,3,disagreement in flag and spectype. z based on ...


conflict ID =  4


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
25,35185766111905627,GALAXY,1.5571,TWL,2,--,--,--,Strange [OII] emission line profile,1.5571,GALAXY,2.0,2,1.557367,0.000313,Strange [OII] emission line profile| Noisy sp...,3,none
75,35185766111905627,GALAXY,1.5571,PNT,1,S,1.5571,GALAXY,Noisy spectra OII doublet is kinda resolved a...,1.5571,GALAXY,2.0,2,1.557367,0.000313,Strange [OII] emission line profile| Noisy sp...,3,none
125,35185766111905627,GALAXY,1.5571,ACE,3,--,1.5579,GALAXY,--,1.5579,GALAXY,2.0,2,1.557367,0.000313,Strange [OII] emission line profile| Noisy sp...,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line in skyline region. z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
25,35185766111905627,-1,0.33.0.dev3890,0,GALAXY,1.5571,TWL,2,--,--,...,48.852668,1.5571,GALAXY,2.0,2,0.000313,1.557367,Strange [OII] emission line profile| Noisy sp...,3,disagreement in flag. broadened but unresolved...
75,35185766111905627,-1,0.33.0.dev3890,0,GALAXY,1.5571,PNT,2,S,1.5571,...,48.852668,1.5571,GALAXY,2.0,2,0.000313,1.557367,Strange [OII] emission line profile| Noisy sp...,3,disagreement in flag. broadened but unresolved...
125,35185766111905627,-1,0.33.0.dev3890,0,GALAXY,1.5571,ACE,2,--,1.5579,...,48.852668,1.5579,GALAXY,2.0,2,0.000313,1.557367,Strange [OII] emission line profile| Noisy sp...,3,disagreement in flag. broadened but unresolved...


conflict ID =  5


In [44]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
28,35185766111906265,QSO,1.6776,TWL,0,RCS,--,--,Very low S/N; calibration issue around 6000-7000,1.6776,QSO,1.0,3,1.6776,0.051689,Very low S/N; calibration issue around 6000-7...,3,none
78,35185766111906265,QSO,1.6776,PNT,0,RCS,--,--,Noisy spectra,1.6776,QSO,1.0,3,1.6776,0.051689,Very low S/N; calibration issue around 6000-7...,3,none
128,35185766111906265,QSO,1.6776,ACE,3,R,1.5460,GALAXY,--,1.5460,GALAXY,1.0,3,1.6776,0.051689,Very low S/N; calibration issue around 6000-7...,3,none


In [45]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5460
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, flag and spectype. z not secure, take VI z with flag 1. not a QSO. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
28,35185766111906265,-1,0.33.0.dev3890,0,QSO,1.6776,TWL,1,RCS,--,...,12.786534,1.546,GALAXY,1.0,3,0.051689,1.6776,Very low S/N; calibration issue around 6000-7...,3,"disagreement in z, flag and spectype. z not se..."
78,35185766111906265,-1,0.33.0.dev3890,0,QSO,1.6776,PNT,1,RCS,--,...,12.786534,1.546,GALAXY,1.0,3,0.051689,1.6776,Very low S/N; calibration issue around 6000-7...,3,"disagreement in z, flag and spectype. z not se..."
128,35185766111906265,-1,0.33.0.dev3890,0,QSO,1.6776,ACE,1,R,1.5460,...,12.786534,1.546,GALAXY,1.0,3,0.051689,1.6776,Very low S/N; calibration issue around 6000-7...,3,"disagreement in z, flag and spectype. z not se..."


conflict ID =  6


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
29,35185766111906679,QSO,1.6833,TWL,0,RCS,--,--,Bad calibration feature,1.6833,QSO,1.0,3,1.6833,0.052646,Bad calibration feature|Noisy spectra| OII in...,3,none
79,35185766111906679,QSO,1.6833,PNT,0,RCS,--,--,Noisy spectra,1.6833,QSO,1.0,3,1.6833,0.052646,Bad calibration feature|Noisy spectra| OII in...,3,none
129,35185766111906679,QSO,1.6833,ACE,3,RC,1.5491,GALAXY,OII in sky lines and MgII in absorption,1.5491,GALAXY,1.0,3,1.6833,0.052646,Bad calibration feature|Noisy spectra| OII in...,3,none


In [49]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5491
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 1/3 inspectors suggests VI z on weak OII on sklylines and MgII. take with flag 2. poor sky subtraction. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
29,35185766111906679,-1,0.33.0.dev3890,0,QSO,1.6833,TWL,2,RCS,--,...,16.844741,1.5491,GALAXY,1.0,3,0.052646,1.6833,Bad calibration feature|Noisy spectra| OII in...,3,disagreement in flag and z. 1/3 inspectors sug...
79,35185766111906679,-1,0.33.0.dev3890,0,QSO,1.6833,PNT,2,RCS,--,...,16.844741,1.5491,GALAXY,1.0,3,0.052646,1.6833,Bad calibration feature|Noisy spectra| OII in...,3,disagreement in flag and z. 1/3 inspectors sug...
129,35185766111906679,-1,0.33.0.dev3890,0,QSO,1.6833,ACE,2,RC,1.5491,...,16.844741,1.5491,GALAXY,1.0,3,0.052646,1.6833,Bad calibration feature|Noisy spectra| OII in...,3,disagreement in flag and z. 1/3 inspectors sug...


conflict ID =  7


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
33,35185766111907627,GALAXY,0.6226,TWL,3,RS,1.1795,--,[OII] at z~1.1795; bad calibration issue,1.1795,GALAXY,2.666667,3,0.6226,0.343215,[OII] at z~1.1795; bad calibration issue| Noi...,3,none
83,35185766111907627,GALAXY,0.6226,PNT,1,S,0.6226,GALAXY,Noisy spectra OIII doublet line is resolved a...,0.6226,GALAXY,2.666667,3,0.6226,0.343215,[OII] at z~1.1795; bad calibration issue| Noi...,3,none
133,35185766111907627,GALAXY,0.6226,ACE,4,--,1.1795,GALAXY,MgII in emission. Redrock confuses OII for OIII.,1.1795,GALAXY,2.666667,3,0.6226,0.343215,[OII] at z~1.1795; bad calibration issue| Noi...,3,none


In [53]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1795
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. 2/3 inspectors give z based on missed OII doublet. Secure. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
33,35185766111907627,-1,0.33.0.dev3890,0,GALAXY,0.6226,TWL,3,RS,1.1795,...,315.327857,1.1795,GALAXY,2.666667,3,0.343215,0.6226,[OII] at z~1.1795; bad calibration issue| Noi...,3,disagreement in z. 2/3 inspectors give z based...
83,35185766111907627,-1,0.33.0.dev3890,0,GALAXY,0.6226,PNT,3,S,0.6226,...,315.327857,1.1795,GALAXY,2.666667,3,0.343215,0.6226,[OII] at z~1.1795; bad calibration issue| Noi...,3,disagreement in z. 2/3 inspectors give z based...
133,35185766111907627,-1,0.33.0.dev3890,0,GALAXY,0.6226,ACE,3,--,1.1795,...,315.327857,1.1795,GALAXY,2.666667,3,0.343215,0.6226,[OII] at z~1.1795; bad calibration issue| Noi...,3,disagreement in z. 2/3 inspectors give z based...


conflict ID =  8


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
34,35185766111908072,QSO,1.7153,TWL,1,S,--,--,--,1.7153,QSO,1.333333,3,1.7153,0.073496,--| Noisy spectra|--,3,none
84,35185766111908072,QSO,1.7153,PNT,0,RCS,--,--,Noisy spectra,1.7153,QSO,1.333333,3,1.7153,0.073496,--| Noisy spectra|--,3,none
134,35185766111908072,QSO,1.7153,ACE,3,--,1.5294,GALAXY,--,1.5294,GALAXY,1.333333,3,1.7153,0.073496,--| Noisy spectra|--,3,none


In [59]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.7153
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. z not secure, take RR answer with flag 1. not a QSO. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
34,35185766111908072,-1,0.33.0.dev3890,0,QSO,1.7153,TWL,1,S,--,...,27.859737,1.7153,QSO,1.333333,3,0.073496,1.7153,--| Noisy spectra|--,3,"disagreement in all. z not secure, take RR ans..."
84,35185766111908072,-1,0.33.0.dev3890,0,QSO,1.7153,PNT,1,RCS,--,...,27.859737,1.7153,QSO,1.333333,3,0.073496,1.7153,--| Noisy spectra|--,3,"disagreement in all. z not secure, take RR ans..."
134,35185766111908072,-1,0.33.0.dev3890,0,QSO,1.7153,ACE,1,--,1.5294,...,27.859737,1.7153,QSO,1.333333,3,0.073496,1.7153,--| Noisy spectra|--,3,"disagreement in all. z not secure, take RR ans..."


conflict ID =  9


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
35,35185766111908276,GALAXY,0.4481,TWL,2,--,1.1521,--,Bad calibration around 5800A; possibly [OII] ...,1.1521,GALAXY,1.666667,3,0.4481,0.486292,Bad calibration around 5800A; possibly [OII] ...,3,none
85,35185766111908276,GALAXY,0.4481,PNT,0,RCS,--,--,Noisy spectra negative spectra,0.4481,GALAXY,1.666667,3,0.4481,0.486292,Bad calibration around 5800A; possibly [OII] ...,3,none
135,35185766111908276,GALAXY,0.4481,ACE,3,RS,1.1523,GALAXY,Continuum mismatch 5800-6300A.,1.1523,GALAXY,1.666667,3,0.4481,0.486292,Bad calibration around 5800A; possibly [OII] ...,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1522
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give same VI z. OII doublet not resolved. potential continuum and weak balmer lines. take solution with mean flag of 2.5. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
35,35185766111908276,-1,0.33.0.dev3890,0,GALAXY,0.4481,TWL,2.5,--,1.1521,...,118.482411,1.1522,GALAXY,1.666667,3,0.486292,0.4481,Bad calibration around 5800A; possibly [OII] ...,3,disagreement in z and flag. 2/3 inspectors giv...
85,35185766111908276,-1,0.33.0.dev3890,0,GALAXY,0.4481,PNT,2.5,RCS,--,...,118.482411,1.1522,GALAXY,1.666667,3,0.486292,0.4481,Bad calibration around 5800A; possibly [OII] ...,3,disagreement in z and flag. 2/3 inspectors giv...
135,35185766111908276,-1,0.33.0.dev3890,0,GALAXY,0.4481,ACE,2.5,RS,1.1523,...,118.482411,1.1522,GALAXY,1.666667,3,0.486292,0.4481,Bad calibration around 5800A; possibly [OII] ...,3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  10


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
39,35185766111909116,QSO,1.6738,TWL,0.0,S,--,--,Bad calibration,1.6738,QSO,1.0,3,1.6738,0.296828,Bad calibration| Noisy spectra|--,3,none
89,35185766111909116,QSO,1.6738,PNT,0.0,RCS,--,--,Noisy spectra,1.6738,QSO,1.0,3,1.6738,0.296828,Bad calibration| Noisy spectra|--,3,none
139,35185766111909116,QSO,1.6738,ACE,3.0,--,1.0618,GALAXY,--,1.0618,GALAXY,1.0,3,1.6738,0.296828,Bad calibration| Noisy spectra|--,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0618
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. one inspector gives VI based on OII doublet. take with flag 3. not a QSO. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
39,35185766111909116,-1,0.33.0.dev3890,0,QSO,1.6738,TWL,3.0,S,--,...,2.247564,1.0618,GALAXY,1.0,3,0.296828,1.6738,Bad calibration| Noisy spectra|--,3,disagreement in all. one inspector gives VI ba...
89,35185766111909116,-1,0.33.0.dev3890,0,QSO,1.6738,PNT,3.0,RCS,--,...,2.247564,1.0618,GALAXY,1.0,3,0.296828,1.6738,Bad calibration| Noisy spectra|--,3,disagreement in all. one inspector gives VI ba...
139,35185766111909116,-1,0.33.0.dev3890,0,QSO,1.6738,ACE,3.0,--,1.0618,...,2.247564,1.0618,GALAXY,1.0,3,0.296828,1.6738,Bad calibration| Noisy spectra|--,3,disagreement in all. one inspector gives VI ba...


conflict ID =  11


In [71]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
3,35185766116098370,GALAXY,0.4881,TWL,2.0,--,--,--,Model fits to [OIII] lines but no [OII] lines...,0.4881,GALAXY,2.666667,2,0.4881,0.0,Model fits to [OIII] lines but no [OII] lines...,3,none
53,35185766116098370,GALAXY,0.4881,PNT,2.0,S,0.4881,GALAXY,Noisy spectra noise is not shown only OII lin...,0.4881,GALAXY,2.666667,2,0.4881,0.0,Model fits to [OIII] lines but no [OII] lines...,3,none
103,35185766116098370,GALAXY,0.4881,ACE,4.0,--,0.4881,GALAXY,Plus strong OII at z=1.0834. Lower redshift s...,0.4881,GALAXY,2.666667,2,0.4881,0.0,Model fits to [OIII] lines but no [OII] lines...,3,none


In [72]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, z is secure - OIII,Ha. Flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
3,35185766116098370,-1,0.33.0.dev3890,0,GALAXY,0.4881,TWL,3.0,--,--,...,400.307992,0.4881,GALAXY,2.666667,2,0.0,0.4881,Model fits to [OIII] lines but no [OII] lines...,3,"flag disagreement, z is secure - OIII,Ha. Flag 3."
53,35185766116098370,-1,0.33.0.dev3890,0,GALAXY,0.4881,PNT,3.0,S,0.4881,...,400.307992,0.4881,GALAXY,2.666667,2,0.0,0.4881,Model fits to [OIII] lines but no [OII] lines...,3,"flag disagreement, z is secure - OIII,Ha. Flag 3."
103,35185766116098370,-1,0.33.0.dev3890,0,GALAXY,0.4881,ACE,3.0,--,0.4881,...,400.307992,0.4881,GALAXY,2.666667,2,0.0,0.4881,Model fits to [OIII] lines but no [OII] lines...,3,"flag disagreement, z is secure - OIII,Ha. Flag 3."


conflict ID =  12


In [75]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
6,35185766116098878,GALAXY,1.1977,TWL,3.0,--,--,--,--,1.1977,GALAXY,2.333333,2,1.1977,0.0,--|Noisy spectra OII doublet is resolved as on...,3,none
56,35185766116098878,GALAXY,1.1977,PNT,1.0,--,1.1977,GALAXY,Noisy spectra OII doublet is resolved as one s...,1.1977,GALAXY,2.333333,2,1.1977,0.0,--|Noisy spectra OII doublet is resolved as on...,3,none
106,35185766116098878,GALAXY,1.1977,ACE,3.0,--,--,--,--,1.1977,GALAXY,2.333333,2,1.1977,0.0,--|Noisy spectra OII doublet is resolved as on...,3,none


In [76]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened OII but continuum is consistent. secure z. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
6,35185766116098878,-1,0.33.0.dev3890,0,GALAXY,1.1977,TWL,3.0,--,--,...,20.582957,1.1977,GALAXY,2.333333,2,0.0,1.1977,--|Noisy spectra OII doublet is resolved as on...,3,disagreement in flag. broadened OII but contin...
56,35185766116098878,-1,0.33.0.dev3890,0,GALAXY,1.1977,PNT,3.0,--,1.1977,...,20.582957,1.1977,GALAXY,2.333333,2,0.0,1.1977,--|Noisy spectra OII doublet is resolved as on...,3,disagreement in flag. broadened OII but contin...
106,35185766116098878,-1,0.33.0.dev3890,0,GALAXY,1.1977,ACE,3.0,--,--,...,20.582957,1.1977,GALAXY,2.333333,2,0.0,1.1977,--|Noisy spectra OII doublet is resolved as on...,3,disagreement in flag. broadened OII but contin...


conflict ID =  13


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
8,35185766116099530,GALAXY,1.2044,TWL,2.0,--,--,--,Strange [OII] profile,1.2044,GALAXY,2.0,2,1.2044,0.0,Strange [OII] profile| Noisy spectra OII doubl...,3,none
58,35185766116099530,GALAXY,1.2044,PNT,1.0,--,1.2044,GALAXY,Noisy spectra OII doublet is observed as one ...,1.2044,GALAXY,2.0,2,1.2044,0.0,Strange [OII] profile| Noisy spectra OII doubl...,3,none
108,35185766116099530,GALAXY,1.2044,ACE,3.0,--,--,--,--,1.2044,GALAXY,2.0,2,1.2044,0.0,Strange [OII] profile| Noisy spectra OII doubl...,3,none


In [80]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. continuum consistent with OII, secure z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
8,35185766116099530,-1,0.33.0.dev3890,0,GALAXY,1.2044,TWL,3.0,--,--,...,196.107786,1.2044,GALAXY,2.0,2,0.0,1.2044,Strange [OII] profile| Noisy spectra OII doubl...,3,disagreement in flag. broadened but unresolved...
58,35185766116099530,-1,0.33.0.dev3890,0,GALAXY,1.2044,PNT,3.0,--,1.2044,...,196.107786,1.2044,GALAXY,2.0,2,0.0,1.2044,Strange [OII] profile| Noisy spectra OII doubl...,3,disagreement in flag. broadened but unresolved...
108,35185766116099530,-1,0.33.0.dev3890,0,GALAXY,1.2044,ACE,3.0,--,--,...,196.107786,1.2044,GALAXY,2.0,2,0.0,1.2044,Strange [OII] profile| Noisy spectra OII doubl...,3,disagreement in flag. broadened but unresolved...


conflict ID =  14


In [83]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
11,35185766116100334,GALAXY,1.6837,TWL,0.0,--,--,--,Low S/N,1.6837,GALAXY,1.0,3,1.6837,0.050788,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,none
61,35185766116100334,GALAXY,1.6837,PNT,0.0,RCS,--,--,Noisy spectra,1.6837,GALAXY,1.0,3,1.6837,0.050788,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,none
111,35185766116100334,GALAXY,1.6837,ACE,3.0,--,1.8200,QSO,"SiIV"","" HeII and CIII]",1.8200,QSO,1.0,3,1.6837,0.050788,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,none


In [84]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6837
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. very poor signal. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
11,35185766116100334,-1,0.33.0.dev3890,0,GALAXY,1.6837,TWL,1.0,--,--,...,8.772847,1.6837,GALAXY,1.0,3,0.050788,1.6837,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,disagreement in all. very poor signal. take RR...
61,35185766116100334,-1,0.33.0.dev3890,0,GALAXY,1.6837,PNT,1.0,RCS,--,...,8.772847,1.6837,GALAXY,1.0,3,0.050788,1.6837,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,disagreement in all. very poor signal. take RR...
111,35185766116100334,-1,0.33.0.dev3890,0,GALAXY,1.6837,ACE,1.0,--,1.8200,...,8.772847,1.6837,GALAXY,1.0,3,0.050788,1.6837,"Low S/N| Noisy spectra|SiIV"","" HeII and CIII]",3,disagreement in all. very poor signal. take RR...


conflict ID =  15


In [87]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
14,35185766116101295,GALAXY,1.3606,TWL,2.0,--,--,--,Strange [OII] emission line profile; being ca...,1.3606,GALAXY,2.0,2,1.3606,0.0,Strange [OII] emission line profile; being ca...,3,none
64,35185766116101295,GALAXY,1.3606,PNT,1.0,--,1.3606,GALAXY,Noisy spectra OII doublet is resolved as one ...,1.3606,GALAXY,2.0,2,1.3606,0.0,Strange [OII] emission line profile; being ca...,3,none
114,35185766116101295,GALAXY,1.3606,ACE,3.0,--,--,--,--,1.3606,GALAXY,2.0,2,1.3606,0.0,Strange [OII] emission line profile; being ca...,3,none


In [88]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. z not secure. poor sky subtraction. set to mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
14,35185766116101295,-1,0.33.0.dev3890,0,GALAXY,1.3606,TWL,2.0,--,--,...,193.174556,1.3606,GALAXY,2.0,2,0.0,1.3606,Strange [OII] emission line profile; being ca...,3,disagreement in flag. broadened but unresolved...
64,35185766116101295,-1,0.33.0.dev3890,0,GALAXY,1.3606,PNT,2.0,--,1.3606,...,193.174556,1.3606,GALAXY,2.0,2,0.0,1.3606,Strange [OII] emission line profile; being ca...,3,disagreement in flag. broadened but unresolved...
114,35185766116101295,-1,0.33.0.dev3890,0,GALAXY,1.3606,ACE,2.0,--,--,...,193.174556,1.3606,GALAXY,2.0,2,0.0,1.3606,Strange [OII] emission line profile; being ca...,3,disagreement in flag. broadened but unresolved...


conflict ID =  16


In [91]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
15,35185766116101682,GALAXY,1.5173,TWL,0.0,R,--,--,Model fits on sky residuals,1.5173,GALAXY,1.0,3,1.5173,0.023994,Model fits on sky residuals|Noisy spectra|--,3,none
65,35185766116101682,GALAXY,1.5173,PNT,0.0,RCS,--,--,Noisy spectra,1.5173,GALAXY,1.0,3,1.5173,0.023994,Model fits on sky residuals|Noisy spectra|--,3,none
115,35185766116101682,GALAXY,1.5173,ACE,3.0,--,1.5777,GALAXY,--,1.5777,GALAXY,1.0,3,1.5173,0.023994,Model fits on sky residuals|Noisy spectra|--,3,none


In [92]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
15,35185766116101682,-1,0.33.0.dev3890,0,GALAXY,1.5173,TWL,1.0,R,--,...,4.51341,1.5173,GALAXY,1.0,3,0.023994,1.5173,Model fits on sky residuals|Noisy spectra|--,3,"flag disagreement, take mean."
65,35185766116101682,-1,0.33.0.dev3890,0,GALAXY,1.5173,PNT,1.0,RCS,--,...,4.51341,1.5173,GALAXY,1.0,3,0.023994,1.5173,Model fits on sky residuals|Noisy spectra|--,3,"flag disagreement, take mean."
115,35185766116101682,-1,0.33.0.dev3890,0,GALAXY,1.5173,ACE,1.0,--,1.5777,...,4.51341,1.5777,GALAXY,1.0,3,0.023994,1.5173,Model fits on sky residuals|Noisy spectra|--,3,"flag disagreement, take mean."


conflict ID =  17


In [95]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185766116102188,GALAXY,0.4432,TWL,0.0,S,--,--,Bad calibration around 5800 A,0.4432,GALAXY,1.333333,4,0.4432,0.762472,Bad calibration around 5800 A| Noisy spectra|...,3,none
93,35185766116102188,GALAXY,0.4432,PNT,0.0,RCS,--,--,Noisy spectra,0.4432,GALAXY,1.333333,4,0.4432,0.762472,Bad calibration around 5800 A| Noisy spectra|...,3,none
143,35185766116102188,GALAXY,0.4432,ACE,4.0,RCS,1.5436,QSO,Continuum mismatch 5800-6300A.,1.5436,QSO,1.333333,4,0.4432,0.762472,Bad calibration around 5800 A| Noisy spectra|...,3,none


In [96]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5436
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector suggests VI z for QSO. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
43,35185766116102188,-1,0.33.0.dev3890,0,GALAXY,0.4432,TWL,2.0,S,--,...,41.77075,1.5436,QSO,1.333333,4,0.762472,0.4432,Bad calibration around 5800 A| Noisy spectra|...,3,one inspector suggests VI z for QSO. take with...
93,35185766116102188,-1,0.33.0.dev3890,0,GALAXY,0.4432,PNT,2.0,RCS,--,...,41.77075,1.5436,QSO,1.333333,4,0.762472,0.4432,Bad calibration around 5800 A| Noisy spectra|...,3,one inspector suggests VI z for QSO. take with...
143,35185766116102188,-1,0.33.0.dev3890,0,GALAXY,0.4432,ACE,2.0,RCS,1.5436,...,41.77075,1.5436,QSO,1.333333,4,0.762472,0.4432,Bad calibration around 5800 A| Noisy spectra|...,3,one inspector suggests VI z for QSO. take with...


conflict ID =  18


In [99]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
44,35185766116102579,GALAXY,0.4464,TWL,2.0,--,0.9930,--,Bad calibration around 5800 A; possible [OII]...,0.9930,GALAXY,1.666667,3,0.4464,0.377904,Bad calibration around 5800 A; possible [OII]...,3,none
94,35185766116102579,GALAXY,0.4464,PNT,0.0,RCS,--,--,Noisy spectra,0.4464,GALAXY,1.666667,3,0.4464,0.377904,Bad calibration around 5800 A; possible [OII]...,3,none
144,35185766116102579,GALAXY,0.4464,ACE,3.0,--,0.9930,GALAXY,Continuum mismatch 5800-6300A.,0.9930,GALAXY,1.666667,3,0.4464,0.377904,Bad calibration around 5800 A; possible [OII]...,3,none


In [100]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9930
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors suggest secure z on OII doublet. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
44,35185766116102579,-1,0.33.0.dev3890,0,GALAXY,0.4464,TWL,3.0,--,0.9930,...,51.3973,0.993,GALAXY,1.666667,3,0.377904,0.4464,Bad calibration around 5800 A; possible [OII]...,3,disagreement in flag and z. 2/3 inspectors sug...
94,35185766116102579,-1,0.33.0.dev3890,0,GALAXY,0.4464,PNT,3.0,RCS,--,...,51.3973,0.993,GALAXY,1.666667,3,0.377904,0.4464,Bad calibration around 5800 A; possible [OII]...,3,disagreement in flag and z. 2/3 inspectors sug...
144,35185766116102579,-1,0.33.0.dev3890,0,GALAXY,0.4464,ACE,3.0,--,0.9930,...,51.3973,0.993,GALAXY,1.666667,3,0.377904,0.4464,Bad calibration around 5800 A; possible [OII]...,3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  19


In [101]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
16,35185766116102675,GALAXY,1.0354,TWL,4.0,--,--,--,--,1.0354,GALAXY,3.333333,2,1.0354,0.0,--| Noisy spectra OII doublet is resolved|--,3,none
66,35185766116102675,GALAXY,1.0354,PNT,2.0,S,1.0354,GALAXY,Noisy spectra OII doublet is resolved,1.0354,GALAXY,3.333333,2,1.0354,0.0,--| Noisy spectra OII doublet is resolved|--,3,none
116,35185766116102675,GALAXY,1.0354,ACE,4.0,--,--,--,--,1.0354,GALAXY,3.333333,2,1.0354,0.0,--| Noisy spectra OII doublet is resolved|--,3,none


In [102]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
16,35185766116102675,-1,0.33.0.dev3890,0,GALAXY,1.0354,TWL,3.333333,--,--,...,80.628124,1.0354,GALAXY,3.333333,2,0.0,1.0354,--| Noisy spectra OII doublet is resolved|--,3,"flag disagreement, take mean."
66,35185766116102675,-1,0.33.0.dev3890,0,GALAXY,1.0354,PNT,3.333333,S,1.0354,...,80.628124,1.0354,GALAXY,3.333333,2,0.0,1.0354,--| Noisy spectra OII doublet is resolved|--,3,"flag disagreement, take mean."
116,35185766116102675,-1,0.33.0.dev3890,0,GALAXY,1.0354,ACE,3.333333,--,--,...,80.628124,1.0354,GALAXY,3.333333,2,0.0,1.0354,--| Noisy spectra OII doublet is resolved|--,3,"flag disagreement, take mean."


conflict ID =  20


In [105]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
45,35185766116102727,QSO,0.8974,TWL,4.0,S,--,GALAXY,Bad calibration,0.8974,GALAXY,3.333333,2,0.8974,0.0,Bad calibration| Noisy spectra some negative ...,3,none
95,35185766116102727,QSO,0.8974,PNT,2.0,S,0.8974,GALAXY,Noisy spectra some negative spectra but OII d...,0.8974,GALAXY,3.333333,2,0.8974,0.0,Bad calibration| Noisy spectra some negative ...,3,none
145,35185766116102727,QSO,0.8974,ACE,4.0,--,--,--,Continuum mismatch 5800-6300A.,0.8974,QSO,3.333333,2,0.8974,0.0,Bad calibration| Noisy spectra some negative ...,3,none


In [106]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. Not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
45,35185766116102727,-1,0.33.0.dev3890,0,QSO,0.8974,TWL,3.333333,S,--,...,292.542698,0.8974,GALAXY,3.333333,2,0.0,0.8974,Bad calibration| Noisy spectra some negative ...,3,"flag disagreement, take mean. Not a QSO."
95,35185766116102727,-1,0.33.0.dev3890,0,QSO,0.8974,PNT,3.333333,S,0.8974,...,292.542698,0.8974,GALAXY,3.333333,2,0.0,0.8974,Bad calibration| Noisy spectra some negative ...,3,"flag disagreement, take mean. Not a QSO."
145,35185766116102727,-1,0.33.0.dev3890,0,QSO,0.8974,ACE,3.333333,--,--,...,292.542698,0.8974,GALAXY,3.333333,2,0.0,0.8974,Bad calibration| Noisy spectra some negative ...,3,"flag disagreement, take mean. Not a QSO."


conflict ID =  21


In [110]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
46,35185766116103427,GALAXY,0.4854,TWL,3.0,RCS,0.9952,GALAXY,Bad calibration around 5800 A; likely [OII] a...,0.9952,GALAXY,3.0,2,0.4854,0.343207,Bad calibration around 5800 A; likely [OII] a...,3,none
96,35185766116103427,GALAXY,0.4854,PNT,2.0,S,0.4854,GALAXY,Noisy spectra but OIII lines are kinda resolve...,0.4854,GALAXY,3.0,2,0.4854,0.343207,Bad calibration around 5800 A; likely [OII] a...,3,none
146,35185766116103427,GALAXY,0.4854,ACE,4.0,--,--,--,Continuum mismatch 5800-6300A.,0.4854,GALAXY,3.0,2,0.4854,0.343207,Bad calibration around 5800 A; likely [OII] a...,3,none


In [111]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9952
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. one clear broadened but unresolved line. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
46,35185766116103427,-1,0.33.0.dev3890,0,GALAXY,0.4854,TWL,2.0,RCS,0.9952,...,217.044105,0.9952,GALAXY,3.0,2,0.343207,0.4854,Bad calibration around 5800 A; likely [OII] a...,3,disagreement in flag and z. one clear broadene...
96,35185766116103427,-1,0.33.0.dev3890,0,GALAXY,0.4854,PNT,2.0,S,0.4854,...,217.044105,0.9952,GALAXY,3.0,2,0.343207,0.4854,Bad calibration around 5800 A; likely [OII] a...,3,disagreement in flag and z. one clear broadene...
146,35185766116103427,-1,0.33.0.dev3890,0,GALAXY,0.4854,ACE,2.0,--,--,...,217.044105,0.9952,GALAXY,3.0,2,0.343207,0.4854,Bad calibration around 5800 A; likely [OII] a...,3,disagreement in flag and z. one clear broadene...


conflict ID =  22


In [114]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
47,35185766116103536,GALAXY,0.4351,TWL,0.0,S,--,--,Bad calibration around 5800 A,0.4351,GALAXY,1.0,3,0.4351,1.205909,Bad calibration around 5800 A| Noisy spectra| ...,3,none
97,35185766116103536,GALAXY,0.4351,PNT,0.0,RCS,--,--,Noisy spectra,0.4351,GALAXY,1.0,3,0.4351,1.205909,Bad calibration around 5800 A| Noisy spectra| ...,3,none
147,35185766116103536,GALAXY,0.4351,ACE,3.0,--,2.1657,GALAXY,Continuum mismatch 5800-6300A. Based on Lyma...,2.1657,GALAXY,1.0,3,0.4351,1.205909,Bad calibration around 5800 A| Noisy spectra| ...,3,none


In [115]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4351
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag.no signal. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
47,35185766116103536,-1,0.33.0.dev3890,0,GALAXY,0.4351,TWL,1.0,S,--,...,1.900895,0.4351,GALAXY,1.0,3,1.205909,0.4351,Bad calibration around 5800 A| Noisy spectra| ...,3,disagreement in flag.no signal. take RR answer...
97,35185766116103536,-1,0.33.0.dev3890,0,GALAXY,0.4351,PNT,1.0,RCS,--,...,1.900895,0.4351,GALAXY,1.0,3,1.205909,0.4351,Bad calibration around 5800 A| Noisy spectra| ...,3,disagreement in flag.no signal. take RR answer...
147,35185766116103536,-1,0.33.0.dev3890,0,GALAXY,0.4351,ACE,1.0,--,2.1657,...,1.900895,0.4351,GALAXY,1.0,3,1.205909,0.4351,Bad calibration around 5800 A| Noisy spectra| ...,3,disagreement in flag.no signal. take RR answer...


conflict ID =  23


In [118]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
150,35185766120292507,GALAXY,1.6826,PNT,0.0,RCS,--,--,Noisy spectra,1.6826,GALAXY,1.0,3,1.6826,0.039808,Noisy spectra|OII in sky lines.|Very low S/N,3,none
200,35185766120292507,GALAXY,1.6826,ACE,3.0,R,1.5799,GALAXY,OII in sky lines.,1.5799,GALAXY,1.0,3,1.6826,0.039808,Noisy spectra|OII in sky lines.|Very low S/N,3,none
250,35185766120292507,GALAXY,1.6826,TWL,0.0,--,--,--,Very low S/N,1.6826,GALAXY,1.0,3,1.6826,0.039808,Noisy spectra|OII in sky lines.|Very low S/N,3,none


In [119]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6826
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. poor sky subtraction. no sigal, z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
150,35185766120292507,-1,0.33.0.dev3890,0,GALAXY,1.6826,PNT,1.0,RCS,--,...,3.256186,1.6826,GALAXY,1.0,3,0.039808,1.6826,Noisy spectra|OII in sky lines.|Very low S/N,3,disagreement in flag and z. poor sky subtracti...
200,35185766120292507,-1,0.33.0.dev3890,0,GALAXY,1.6826,ACE,1.0,R,1.5799,...,3.256186,1.6826,GALAXY,1.0,3,0.039808,1.6826,Noisy spectra|OII in sky lines.|Very low S/N,3,disagreement in flag and z. poor sky subtracti...
250,35185766120292507,-1,0.33.0.dev3890,0,GALAXY,1.6826,TWL,1.0,--,--,...,3.256186,1.6826,GALAXY,1.0,3,0.039808,1.6826,Noisy spectra|OII in sky lines.|Very low S/N,3,disagreement in flag and z. poor sky subtracti...


conflict ID =  24


In [122]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
172,35185766120294735,GALAXY,1.0361,PNT,1.0,S,1.0361,GALAXY,Noisy spectra OII doublet is kinda resolved as...,1.0361,GALAXY,2.333333,3,1.035767,0.000491,Noisy spectra OII doublet is kinda resolved as...,3,none
222,35185766120294735,GALAXY,1.0361,ACE,4.0,--,--,--,--,1.0361,GALAXY,2.333333,3,1.035767,0.000491,Noisy spectra OII doublet is kinda resolved as...,3,none
272,35185766120294735,GALAXY,1.0361,TWL,2.0,--,1.0351,--,Possible [OII] doublet next to sky lines; str...,1.0351,GALAXY,2.333333,3,1.035767,0.000491,Noisy spectra OII doublet is kinda resolved as...,3,none


In [123]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
172,35185766120294735,-1,0.33.0.dev3890,0,GALAXY,1.0361,PNT,2.333333,S,1.0361,...,88.13788,1.0361,GALAXY,2.333333,3,0.000491,1.035767,Noisy spectra OII doublet is kinda resolved as...,3,"flag disagreement, take mean."
222,35185766120294735,-1,0.33.0.dev3890,0,GALAXY,1.0361,ACE,2.333333,--,--,...,88.13788,1.0361,GALAXY,2.333333,3,0.000491,1.035767,Noisy spectra OII doublet is kinda resolved as...,3,"flag disagreement, take mean."
272,35185766120294735,-1,0.33.0.dev3890,0,GALAXY,1.0361,TWL,2.333333,--,1.0351,...,88.13788,1.0351,GALAXY,2.333333,3,0.000491,1.035767,Noisy spectra OII doublet is kinda resolved as...,3,"flag disagreement, take mean."


conflict ID =  25


In [126]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
152,35185766120295131,GALAXY,0.4612,PNT,0.0,RCS,--,--,Noisy spectra,0.4612,GALAXY,1.333333,3,0.4612,0.137763,Noisy spectra| Based on OII and MgII in emiss...,3,none
202,35185766120295131,GALAXY,0.4612,ACE,3.0,R,0.6625,GALAXY,Based on OII and MgII in emission.,0.6625,GALAXY,1.333333,3,0.4612,0.137763,Noisy spectra| Based on OII and MgII in emiss...,3,none
252,35185766120295131,GALAXY,0.4612,TWL,1.0,--,--,--,Continuum in blue; no line; model fits [OII] ...,0.4612,GALAXY,1.333333,3,0.4612,0.137763,Noisy spectra| Based on OII and MgII in emiss...,3,none


In [127]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4612
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. too noisy. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
152,35185766120295131,-1,0.33.0.dev3890,0,GALAXY,0.4612,PNT,1.0,RCS,--,...,3.742199,0.4612,GALAXY,1.333333,3,0.137763,0.4612,Noisy spectra| Based on OII and MgII in emiss...,3,disagreement in z and flag. too noisy. take RR...
202,35185766120295131,-1,0.33.0.dev3890,0,GALAXY,0.4612,ACE,1.0,R,0.6625,...,3.742199,0.4612,GALAXY,1.333333,3,0.137763,0.4612,Noisy spectra| Based on OII and MgII in emiss...,3,disagreement in z and flag. too noisy. take RR...
252,35185766120295131,-1,0.33.0.dev3890,0,GALAXY,0.4612,TWL,1.0,--,--,...,3.742199,0.4612,GALAXY,1.333333,3,0.137763,0.4612,Noisy spectra| Based on OII and MgII in emiss...,3,disagreement in z and flag. too noisy. take RR...


conflict ID =  26


In [130]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
176,35185766120296240,GALAXY,1.3879,PNT,2.0,S,1.3879,GALAXY,Noisy spectra OII doublet is kinda resolved,1.3879,GALAXY,2.666667,2,1.3879,0.0,Noisy spectra OII doublet is kinda resolved|-...,3,none
226,35185766120296240,GALAXY,1.3879,ACE,4.0,--,--,--,--,1.3879,GALAXY,2.666667,2,1.3879,0.0,Noisy spectra OII doublet is kinda resolved|-...,3,none
276,35185766120296240,GALAXY,1.3879,TWL,2.0,--,--,--,Possible [OII] emission next to sky residuals,1.3879,GALAXY,2.666667,2,1.3879,0.0,Noisy spectra OII doublet is kinda resolved|-...,3,none


In [131]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. broadened but unresolved line. continuum consistent with OII. poor sky subtraction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
176,35185766120296240,-1,0.33.0.dev3890,0,GALAXY,1.3879,PNT,2.666667,S,1.3879,...,29.162031,1.3879,GALAXY,2.666667,2,0.0,1.3879,Noisy spectra OII doublet is kinda resolved|-...,3,"flag disagreement, take mean. broadened but un..."
226,35185766120296240,-1,0.33.0.dev3890,0,GALAXY,1.3879,ACE,2.666667,--,--,...,29.162031,1.3879,GALAXY,2.666667,2,0.0,1.3879,Noisy spectra OII doublet is kinda resolved|-...,3,"flag disagreement, take mean. broadened but un..."
276,35185766120296240,-1,0.33.0.dev3890,0,GALAXY,1.3879,TWL,2.666667,--,--,...,29.162031,1.3879,GALAXY,2.666667,2,0.0,1.3879,Noisy spectra OII doublet is kinda resolved|-...,3,"flag disagreement, take mean. broadened but un..."


conflict ID =  27


In [134]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
154,35185766120296280,GALAXY,1.4988,PNT,0.0,RCS,--,--,Noisy spectra,1.4988,GALAXY,1.0,3,1.4988,1.184648,Noisy spectra|Based on OII and Halpha|model f...,3,none
204,35185766120296280,GALAXY,1.4988,ACE,3.0,--,0.1438,GALAXY,Based on OII and Halpha,0.1438,GALAXY,1.0,3,1.4988,1.184648,Noisy spectra|Based on OII and Halpha|model f...,3,none
254,35185766120296280,GALAXY,1.4988,TWL,0.0,--,--,--,model fits sky residuals ; low S/N,1.4988,GALAXY,1.0,3,1.4988,1.184648,Noisy spectra|Based on OII and Halpha|model f...,3,none


In [135]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4988
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure z. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
154,35185766120296280,-1,0.33.0.dev3890,0,GALAXY,1.4988,PNT,1.0,RCS,--,...,29.929197,1.4988,GALAXY,1.0,3,1.184648,1.4988,Noisy spectra|Based on OII and Halpha|model f...,3,disagreement in flag and z. no secure z. take ...
204,35185766120296280,-1,0.33.0.dev3890,0,GALAXY,1.4988,ACE,1.0,--,0.1438,...,29.929197,1.4988,GALAXY,1.0,3,1.184648,1.4988,Noisy spectra|Based on OII and Halpha|model f...,3,disagreement in flag and z. no secure z. take ...
254,35185766120296280,-1,0.33.0.dev3890,0,GALAXY,1.4988,TWL,1.0,--,--,...,29.929197,1.4988,GALAXY,1.0,3,1.184648,1.4988,Noisy spectra|Based on OII and Halpha|model f...,3,disagreement in flag and z. no secure z. take ...


conflict ID =  28


In [138]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
180,35185766120297337,QSO,1.0603,PNT,0.0,RCS,--,--,Noisy spectra,1.0603,QSO,1.333333,4,1.0603,0.662088,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,none
230,35185766120297337,QSO,1.0603,ACE,4.0,R,2.4244,GALAXY,"Lyman alpha"","" NV and CIV",2.4244,GALAXY,1.333333,4,1.0603,0.662088,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,none
280,35185766120297337,QSO,1.0603,TWL,0.0,--,--,--,Low S/N,1.0603,QSO,1.333333,4,1.0603,0.662088,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,none


In [139]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0603
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. noisy spectra, no secure z. not a QSO. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
180,35185766120297337,-1,0.33.0.dev3890,0,QSO,1.0603,PNT,1.0,RCS,--,...,4.836573,1.0603,GALAXY,1.333333,4,0.662088,1.0603,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,"disagreement in all. noisy spectra, no secure ..."
230,35185766120297337,-1,0.33.0.dev3890,0,QSO,1.0603,ACE,1.0,R,2.4244,...,4.836573,1.0603,GALAXY,1.333333,4,0.662088,1.0603,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,"disagreement in all. noisy spectra, no secure ..."
280,35185766120297337,-1,0.33.0.dev3890,0,QSO,1.0603,TWL,1.0,--,--,...,4.836573,1.0603,GALAXY,1.333333,4,0.662088,1.0603,"Noisy spectra| Lyman alpha"","" NV and CIV|Low...",3,"disagreement in all. noisy spectra, no secure ..."


conflict ID =  29


In [142]:
#copy this text to a new cell to display the conflict
conflict_id= 30
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
181,35185766120297470,GALAXY,0.9657,PNT,1.0,S,0.9657,GALAXY,Interesting Noisy spectra OII doublet is not r...,0.9657,GALAXY,1.333333,3,0.9657,0.0,Interesting Noisy spectra OII doublet is not r...,3,none
231,35185766120297470,GALAXY,0.9657,ACE,3.0,--,--,--,--,0.9657,GALAXY,1.333333,3,0.9657,0.0,Interesting Noisy spectra OII doublet is not r...,3,none
281,35185766120297470,GALAXY,0.9657,TWL,0.0,--,--,--,Low S/N,0.9657,GALAXY,1.333333,3,0.9657,0.0,Interesting Noisy spectra OII doublet is not r...,3,none


In [143]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
181,35185766120297470,-1,0.33.0.dev3890,0,GALAXY,0.9657,PNT,1.333333,S,0.9657,...,5.99092,0.9657,GALAXY,1.333333,3,0.0,0.9657,Interesting Noisy spectra OII doublet is not r...,3,"flag disagreement, take mean."
231,35185766120297470,-1,0.33.0.dev3890,0,GALAXY,0.9657,ACE,1.333333,--,--,...,5.99092,0.9657,GALAXY,1.333333,3,0.0,0.9657,Interesting Noisy spectra OII doublet is not r...,3,"flag disagreement, take mean."
281,35185766120297470,-1,0.33.0.dev3890,0,GALAXY,0.9657,TWL,1.333333,--,--,...,5.99092,0.9657,GALAXY,1.333333,3,0.0,0.9657,Interesting Noisy spectra OII doublet is not r...,3,"flag disagreement, take mean."


conflict ID =  30


In [146]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
157,35185766124486990,GALAXY,1.1357,PNT,2.0,S,1.1357,GALAXY,Noisy spectra OII doublet is kinda resolve du...,1.1357,GALAXY,2.666667,2,1.1357,0.0,Noisy spectra OII doublet is kinda resolve du...,3,none
207,35185766124486990,GALAXY,1.1357,ACE,4.0,--,--,--,--,1.1357,GALAXY,2.666667,2,1.1357,0.0,Noisy spectra OII doublet is kinda resolve du...,3,none
257,35185766124486990,GALAXY,1.1357,TWL,2.0,--,--,--,possible emission line feature next to strong ...,1.1357,GALAXY,2.666667,2,1.1357,0.0,Noisy spectra OII doublet is kinda resolve du...,3,none


In [147]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. OII doublet heavily compromised by skyline. not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
157,35185766124486990,-1,0.33.0.dev3890,0,GALAXY,1.1357,PNT,2.0,S,1.1357,...,51.541619,1.1357,GALAXY,2.666667,2,0.0,1.1357,Noisy spectra OII doublet is kinda resolve du...,3,flag disagreement. OII doublet heavily comprom...
207,35185766124486990,-1,0.33.0.dev3890,0,GALAXY,1.1357,ACE,2.0,--,--,...,51.541619,1.1357,GALAXY,2.666667,2,0.0,1.1357,Noisy spectra OII doublet is kinda resolve du...,3,flag disagreement. OII doublet heavily comprom...
257,35185766124486990,-1,0.33.0.dev3890,0,GALAXY,1.1357,TWL,2.0,--,--,...,51.541619,1.1357,GALAXY,2.666667,2,0.0,1.1357,Noisy spectra OII doublet is kinda resolve du...,3,flag disagreement. OII doublet heavily comprom...


conflict ID =  31


In [150]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
158,35185766124487574,GALAXY,0.4632,PNT,0.0,RCS,--,--,Noisy spectra,0.4632,GALAXY,1.0,3,0.4632,1.064721,Noisy spectra|Based on CIV in emission and Mg...,3,none
208,35185766124487574,GALAXY,0.4632,ACE,3.0,--,2.0211,GALAXY,Based on CIV in emission and MgII in absorption.,2.0211,GALAXY,1.0,3,0.4632,1.064721,Noisy spectra|Based on CIV in emission and Mg...,3,none
258,35185766124487574,GALAXY,0.4632,TWL,0.0,--,--,--,Blue continuum; no line,0.4632,GALAXY,1.0,3,0.4632,1.064721,Noisy spectra|Based on CIV in emission and Mg...,3,none


In [151]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4632
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'diagreement in flag and z. no secure z. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
158,35185766124487574,-1,0.33.0.dev3890,0,GALAXY,0.4632,PNT,1.0,RCS,--,...,11.641927,0.4632,GALAXY,1.0,3,1.064721,0.4632,Noisy spectra|Based on CIV in emission and Mg...,3,diagreement in flag and z. no secure z. take R...
208,35185766124487574,-1,0.33.0.dev3890,0,GALAXY,0.4632,ACE,1.0,--,2.0211,...,11.641927,0.4632,GALAXY,1.0,3,1.064721,0.4632,Noisy spectra|Based on CIV in emission and Mg...,3,diagreement in flag and z. no secure z. take R...
258,35185766124487574,-1,0.33.0.dev3890,0,GALAXY,0.4632,TWL,1.0,--,--,...,11.641927,0.4632,GALAXY,1.0,3,1.064721,0.4632,Noisy spectra|Based on CIV in emission and Mg...,3,diagreement in flag and z. no secure z. take R...


conflict ID =  32


In [154]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
189,35185766124488456,GALAXY,1.1028,PNT,2.0,S,1.1028,GALAXY,Noisy spectra OII doublet is resolved as one...,1.1028,GALAXY,2.666667,2,1.103033,0.000333,Noisy spectra OII doublet is resolved as one...,3,none
239,35185766124488456,GALAXY,1.1028,ACE,4.0,--,1.1035,GALAXY,--,1.1035,GALAXY,2.666667,2,1.103033,0.000333,Noisy spectra OII doublet is resolved as one...,3,none
289,35185766124488456,GALAXY,1.1028,TWL,2.0,--,--,--,Strange [OII] emission profile; being conserv...,1.1028,GALAXY,2.666667,2,1.103033,0.000333,Noisy spectra OII doublet is resolved as one...,3,none


In [155]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'broadened but unresolved line. not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
189,35185766124488456,-1,0.33.0.dev3890,0,GALAXY,1.1028,PNT,2.0,S,1.1028,...,48.023399,1.1028,GALAXY,2.666667,2,0.000333,1.103033,Noisy spectra OII doublet is resolved as one...,3,broadened but unresolved line. not secure. poo...
239,35185766124488456,-1,0.33.0.dev3890,0,GALAXY,1.1028,ACE,2.0,--,1.1035,...,48.023399,1.1035,GALAXY,2.666667,2,0.000333,1.103033,Noisy spectra OII doublet is resolved as one...,3,broadened but unresolved line. not secure. poo...
289,35185766124488456,-1,0.33.0.dev3890,0,GALAXY,1.1028,TWL,2.0,--,--,...,48.023399,1.1028,GALAXY,2.666667,2,0.000333,1.103033,Noisy spectra OII doublet is resolved as one...,3,broadened but unresolved line. not secure. poo...


conflict ID =  33


In [158]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
160,35185766124488648,GALAXY,1.2857,PNT,2.0,S,1.2857,GALAXY,Noisy spectra OII doublet is resolved as one ...,1.2857,GALAXY,3.0,2,1.2857,0.0,Noisy spectra OII doublet is resolved as one ...,3,none
210,35185766124488648,GALAXY,1.2857,ACE,3.0,--,--,--,--,1.2857,GALAXY,3.0,2,1.2857,0.0,Noisy spectra OII doublet is resolved as one ...,3,none
260,35185766124488648,GALAXY,1.2857,TWL,4.0,--,--,--,[OII] doublet ; balmer absorption,1.2857,GALAXY,3.0,2,1.2857,0.0,Noisy spectra OII doublet is resolved as one ...,3,none


In [159]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
160,35185766124488648,-1,0.33.0.dev3890,0,GALAXY,1.2857,PNT,3.0,S,1.2857,...,105.295169,1.2857,GALAXY,3.0,2,0.0,1.2857,Noisy spectra OII doublet is resolved as one ...,3,"flag disagreement, take mean."
210,35185766124488648,-1,0.33.0.dev3890,0,GALAXY,1.2857,ACE,3.0,--,--,...,105.295169,1.2857,GALAXY,3.0,2,0.0,1.2857,Noisy spectra OII doublet is resolved as one ...,3,"flag disagreement, take mean."
260,35185766124488648,-1,0.33.0.dev3890,0,GALAXY,1.2857,TWL,3.0,--,--,...,105.295169,1.2857,GALAXY,3.0,2,0.0,1.2857,Noisy spectra OII doublet is resolved as one ...,3,"flag disagreement, take mean."


conflict ID =  34


In [163]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
163,35185766124490426,GALAXY,1.438,PNT,3.0,S,1.4380,GALAXY,Noisy spectra OII doublet is resolved,1.438,GALAXY,3.0,2,1.438,0.0,Noisy spectra OII doublet is resolved|--| Pos...,3,none
213,35185766124490426,GALAXY,1.438,ACE,4.0,--,--,--,--,1.438,GALAXY,3.0,2,1.438,0.0,Noisy spectra OII doublet is resolved|--| Pos...,3,none
263,35185766124490426,GALAXY,1.438,TWL,2.0,--,--,--,Possible [OII] emission next to sky residuals...,1.438,GALAXY,3.0,2,1.438,0.0,Noisy spectra OII doublet is resolved|--| Pos...,3,none


In [164]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. Poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
163,35185766124490426,-1,0.33.0.dev3890,0,GALAXY,1.438,PNT,3.0,S,1.4380,...,44.896432,1.438,GALAXY,3.0,2,0.0,1.438,Noisy spectra OII doublet is resolved|--| Pos...,3,"flag disagreement, take mean. Poor sky subtrac..."
213,35185766124490426,-1,0.33.0.dev3890,0,GALAXY,1.438,ACE,3.0,--,--,...,44.896432,1.438,GALAXY,3.0,2,0.0,1.438,Noisy spectra OII doublet is resolved|--| Pos...,3,"flag disagreement, take mean. Poor sky subtrac..."
263,35185766124490426,-1,0.33.0.dev3890,0,GALAXY,1.438,TWL,3.0,--,--,...,44.896432,1.438,GALAXY,3.0,2,0.0,1.438,Noisy spectra OII doublet is resolved|--| Pos...,3,"flag disagreement, take mean. Poor sky subtrac..."


conflict ID =  35


In [167]:
#copy this text to a new cell to display the conflict
conflict_id= 36
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
193,35185766124490531,GALAXY,1.0327,PNT,3.0,S,1.0327,GALAXY,Noisy spectra OII doublet is resolved as one ...,1.0327,GALAXY,3.0,2,1.0327,0.0,Noisy spectra OII doublet is resolved as one ...,3,none
243,35185766124490531,GALAXY,1.0327,ACE,4.0,--,--,--,--,1.0327,GALAXY,3.0,2,1.0327,0.0,Noisy spectra OII doublet is resolved as one ...,3,none
293,35185766124490531,GALAXY,1.0327,TWL,2.0,--,--,--,Strange [OII] profile; being conservative,1.0327,GALAXY,3.0,2,1.0327,0.0,Noisy spectra OII doublet is resolved as one ...,3,none


In [168]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. change in continuum suggests OII. take z with mean flag 3. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
193,35185766124490531,-1,0.33.0.dev3890,0,GALAXY,1.0327,PNT,3.0,S,1.0327,...,176.280089,1.0327,GALAXY,3.0,2,0.0,1.0327,Noisy spectra OII doublet is resolved as one ...,3,disagreement in flag. broadened but unresolved...
243,35185766124490531,-1,0.33.0.dev3890,0,GALAXY,1.0327,ACE,3.0,--,--,...,176.280089,1.0327,GALAXY,3.0,2,0.0,1.0327,Noisy spectra OII doublet is resolved as one ...,3,disagreement in flag. broadened but unresolved...
293,35185766124490531,-1,0.33.0.dev3890,0,GALAXY,1.0327,TWL,3.0,--,--,...,176.280089,1.0327,GALAXY,3.0,2,0.0,1.0327,Noisy spectra OII doublet is resolved as one ...,3,disagreement in flag. broadened but unresolved...


conflict ID =  36


In [173]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
164,35185766124490928,GALAXY,0.525,PNT,0.0,RCS,--,--,Noisy spectra,0.5250,GALAXY,1.666667,4,0.525,1.059475,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,none
214,35185766124490928,GALAXY,0.525,ACE,4.0,--,2.1407,GALAXY,"From narrow Lyman alpha"","" NV and CIV.",2.1407,GALAXY,1.666667,4,0.525,1.059475,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,none
264,35185766124490928,GALAXY,0.525,TWL,1.0,--,--,--,some continuum; no line,0.5250,GALAXY,1.666667,4,0.525,1.059475,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,none


In [174]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.525
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. unconvincing VI z. take RR z with flag 1. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
164,35185766124490928,-1,0.33.0.dev3890,0,GALAXY,0.525,PNT,1.0,RCS,--,...,15.37466,0.525,GALAXY,1.666667,4,1.059475,0.525,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,disagreement in z and flag. unconvincing VI z....
214,35185766124490928,-1,0.33.0.dev3890,0,GALAXY,0.525,ACE,1.0,--,2.1407,...,15.37466,0.525,GALAXY,1.666667,4,1.059475,0.525,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,disagreement in z and flag. unconvincing VI z....
264,35185766124490928,-1,0.33.0.dev3890,0,GALAXY,0.525,TWL,1.0,--,--,...,15.37466,0.525,GALAXY,1.666667,4,1.059475,0.525,"Noisy spectra| From narrow Lyman alpha"","" NV ...",3,disagreement in z and flag. unconvincing VI z....


conflict ID =  37


In [177]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
194,35185766124490981,GALAXY,1.3773,PNT,3.0,--,1.3773,GALAXY,Noisy spectra OII doublet is resolved as one...,1.3773,GALAXY,3.0,2,1.3773,0.0,Noisy spectra OII doublet is resolved as one...,3,none
244,35185766124490981,GALAXY,1.3773,ACE,4.0,--,--,--,--,1.3773,GALAXY,3.0,2,1.3773,0.0,Noisy spectra OII doublet is resolved as one...,3,none
294,35185766124490981,GALAXY,1.3773,TWL,2.0,--,--,--,Possible [OII] emission with strange profile;...,1.3773,GALAXY,3.0,2,1.3773,0.0,Noisy spectra OII doublet is resolved as one...,3,none


In [178]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, broadened but unresolved line. change in continuum consistent with OII. set mean flag of 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
194,35185766124490981,-1,0.33.0.dev3890,0,GALAXY,1.3773,PNT,3.0,--,1.3773,...,377.043435,1.3773,GALAXY,3.0,2,0.0,1.3773,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."
244,35185766124490981,-1,0.33.0.dev3890,0,GALAXY,1.3773,ACE,3.0,--,--,...,377.043435,1.3773,GALAXY,3.0,2,0.0,1.3773,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."
294,35185766124490981,-1,0.33.0.dev3890,0,GALAXY,1.3773,TWL,3.0,--,--,...,377.043435,1.3773,GALAXY,3.0,2,0.0,1.3773,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."


conflict ID =  38


In [181]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
196,35185766124491408,GALAXY,1.2835,PNT,1.0,S,1.2835,GALAXY,INTERESTING Noisy spectra OII doublet is reso...,1.2835,GALAXY,2.333333,2,1.2835,0.0,INTERESTING Noisy spectra OII doublet is reso...,3,none
246,35185766124491408,GALAXY,1.2835,ACE,3.0,--,--,--,--,1.2835,GALAXY,2.333333,2,1.2835,0.0,INTERESTING Noisy spectra OII doublet is reso...,3,none
296,35185766124491408,GALAXY,1.2835,TWL,3.0,--,--,--,triple peak [OII] emission,1.2835,GALAXY,2.333333,2,1.2835,0.0,INTERESTING Noisy spectra OII doublet is reso...,3,none


In [182]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, broadened but unresolved line. change in continuum consistent with OII. set flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
196,35185766124491408,-1,0.33.0.dev3890,0,GALAXY,1.2835,PNT,3.0,S,1.2835,...,204.286516,1.2835,GALAXY,2.333333,2,0.0,1.2835,INTERESTING Noisy spectra OII doublet is reso...,3,"disagreement in flag, broadened but unresolved..."
246,35185766124491408,-1,0.33.0.dev3890,0,GALAXY,1.2835,ACE,3.0,--,--,...,204.286516,1.2835,GALAXY,2.333333,2,0.0,1.2835,INTERESTING Noisy spectra OII doublet is reso...,3,"disagreement in flag, broadened but unresolved..."
296,35185766124491408,-1,0.33.0.dev3890,0,GALAXY,1.2835,TWL,3.0,--,--,...,204.286516,1.2835,GALAXY,2.333333,2,0.0,1.2835,INTERESTING Noisy spectra OII doublet is reso...,3,"disagreement in flag, broadened but unresolved..."


conflict ID =  39


In [185]:
#copy this text to a new cell to display the conflict
conflict_id= 40
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
197,35185766124491740,GALAXY,1.403,PNT,3.0,--,1.4030,GALAXY,Noisy spectra OII doublet is resolved as one...,1.403,GALAXY,3.0,2,1.403,0.0,Noisy spectra OII doublet is resolved as one...,3,none
247,35185766124491740,GALAXY,1.403,ACE,4.0,--,--,--,MgII in absorption,1.403,GALAXY,3.0,2,1.403,0.0,Noisy spectra OII doublet is resolved as one...,3,none
297,35185766124491740,GALAXY,1.403,TWL,2.0,--,--,--,[OII] emission next to a skyline; being conse...,1.403,GALAXY,3.0,2,1.403,0.0,Noisy spectra OII doublet is resolved as one...,3,none


In [186]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, broadened but unresolved line. change in continuum consistent with OII. set to the mean flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
197,35185766124491740,-1,0.33.0.dev3890,0,GALAXY,1.403,PNT,3.0,--,1.4030,...,292.762392,1.403,GALAXY,3.0,2,0.0,1.403,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."
247,35185766124491740,-1,0.33.0.dev3890,0,GALAXY,1.403,ACE,3.0,--,--,...,292.762392,1.403,GALAXY,3.0,2,0.0,1.403,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."
297,35185766124491740,-1,0.33.0.dev3890,0,GALAXY,1.403,TWL,3.0,--,--,...,292.762392,1.403,GALAXY,3.0,2,0.0,1.403,Noisy spectra OII doublet is resolved as one...,3,"disagreement in flag, broadened but unresolved..."


conflict ID =  40


In [189]:
#copy this text to a new cell to display the conflict
conflict_id= 41
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35185766128685429,GALAXY,1.385,ACE,3.0,--,1.3842,GALAXY,OII on sky line,1.3842,GALAXY,1.0,3,1.384733,0.000336,OII on sky line|model fits sky residuals| No...,3,none
353,35185766128685429,GALAXY,1.385,TWL,0.0,--,--,--,model fits sky residuals,1.3850,GALAXY,1.0,3,1.384733,0.000336,OII on sky line|model fits sky residuals| No...,3,none
403,35185766128685429,GALAXY,1.385,PNT,0.0,RCS,--,--,Noisy spectra OII doublet is resolved as one...,1.3850,GALAXY,1.0,3,1.384733,0.000336,OII on sky line|model fits sky residuals| No...,3,none


In [190]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35185766128685429,-1,0.33.0.dev3890,0,GALAXY,1.385,ACE,1.0,--,1.3842,...,21.274909,1.3842,GALAXY,1.0,3,0.000336,1.384733,OII on sky line|model fits sky residuals| No...,3,"flag disagreement, take mean."
353,35185766128685429,-1,0.33.0.dev3890,0,GALAXY,1.385,TWL,1.0,--,--,...,21.274909,1.3850,GALAXY,1.0,3,0.000336,1.384733,OII on sky line|model fits sky residuals| No...,3,"flag disagreement, take mean."
403,35185766128685429,-1,0.33.0.dev3890,0,GALAXY,1.385,PNT,1.0,RCS,--,...,21.274909,1.3850,GALAXY,1.0,3,0.000336,1.384733,OII on sky line|model fits sky residuals| No...,3,"flag disagreement, take mean."


conflict ID =  41


In [193]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
310,35185766132877398,GALAXY,1.5299,ACE,4.0,--,--,GALAXY,MgII in emission.,1.5299,GALAXY,2.0,4,1.5299,0.0,MgII in emission.|possible [OII] emission in ...,3,none
360,35185766132877398,GALAXY,1.5299,TWL,2.0,--,--,--,possible [OII] emission in sky line regions,1.5299,GALAXY,2.0,4,1.5299,0.0,MgII in emission.|possible [OII] emission in ...,3,none
410,35185766132877398,GALAXY,1.5299,PNT,0.0,RCS,--,--,Noisy spectra,1.5299,GALAXY,2.0,4,1.5299,0.0,MgII in emission.|possible [OII] emission in ...,3,none


In [194]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
310,35185766132877398,-1,0.33.0.dev3890,0,GALAXY,1.5299,ACE,2.0,--,--,...,28.532545,1.5299,GALAXY,2.0,4,0.0,1.5299,MgII in emission.|possible [OII] emission in ...,3,"flag disagreement, take mean."
360,35185766132877398,-1,0.33.0.dev3890,0,GALAXY,1.5299,TWL,2.0,--,--,...,28.532545,1.5299,GALAXY,2.0,4,0.0,1.5299,MgII in emission.|possible [OII] emission in ...,3,"flag disagreement, take mean."
410,35185766132877398,-1,0.33.0.dev3890,0,GALAXY,1.5299,PNT,2.0,RCS,--,...,28.532545,1.5299,GALAXY,2.0,4,0.0,1.5299,MgII in emission.|possible [OII] emission in ...,3,"flag disagreement, take mean."


conflict ID =  42


In [197]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
333,35185766137071522,GALAXY,1.5321,ACE,3.0,--,--,--,--,1.5321,GALAXY,1.666667,3,1.5321,0.0,--| Possible [OII] doublet around sky lines| ...,3,none
383,35185766137071522,GALAXY,1.5321,TWL,2.0,--,--,--,Possible [OII] doublet around sky lines,1.5321,GALAXY,1.666667,3,1.5321,0.0,--| Possible [OII] doublet around sky lines| ...,3,none
433,35185766137071522,GALAXY,1.5321,PNT,0.0,RCS,--,--,Noisy spectra OII doublet is kinda resolved ...,1.5321,GALAXY,1.666667,3,1.5321,0.0,--| Possible [OII] doublet around sky lines| ...,3,none


In [198]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
333,35185766137071522,-1,0.33.0.dev3890,0,GALAXY,1.5321,ACE,1.666667,--,--,...,47.254198,1.5321,GALAXY,1.666667,3,0.0,1.5321,--| Possible [OII] doublet around sky lines| ...,3,"flag disagreement, take mean."
383,35185766137071522,-1,0.33.0.dev3890,0,GALAXY,1.5321,TWL,1.666667,--,--,...,47.254198,1.5321,GALAXY,1.666667,3,0.0,1.5321,--| Possible [OII] doublet around sky lines| ...,3,"flag disagreement, take mean."
433,35185766137071522,-1,0.33.0.dev3890,0,GALAXY,1.5321,PNT,1.666667,RCS,--,...,47.254198,1.5321,GALAXY,1.666667,3,0.0,1.5321,--| Possible [OII] doublet around sky lines| ...,3,"flag disagreement, take mean."


conflict ID =  43


In [201]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
335,35185766137071782,GALAXY,0.2397,ACE,4.0,--,0.2397,GALAXY,And a possible set of other lines that I coul...,0.2397,GALAXY,2.333333,3,0.2397,0.0,And a possible set of other lines that I coul...,3,none
385,35185766137071782,GALAXY,0.2397,TWL,1.0,--,--,--,No obvious strong features,0.2397,GALAXY,2.333333,3,0.2397,0.0,And a possible set of other lines that I coul...,3,none
435,35185766137071782,GALAXY,0.2397,PNT,2.0,--,0.2397,GALAXY,Noisy spectra only Ha is resolved at λobs~8150A,0.2397,GALAXY,2.333333,3,0.2397,0.0,And a possible set of other lines that I coul...,3,none


In [202]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2397
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Ha and OIII secure z. flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
335,35185766137071782,-1,0.33.0.dev3890,0,GALAXY,0.2397,ACE,3.0,--,0.2397,...,100.902124,0.2397,GALAXY,2.333333,3,0.0,0.2397,And a possible set of other lines that I coul...,3,disagreement in flag. Ha and OIII secure z. fl...
385,35185766137071782,-1,0.33.0.dev3890,0,GALAXY,0.2397,TWL,3.0,--,--,...,100.902124,0.2397,GALAXY,2.333333,3,0.0,0.2397,And a possible set of other lines that I coul...,3,disagreement in flag. Ha and OIII secure z. fl...
435,35185766137071782,-1,0.33.0.dev3890,0,GALAXY,0.2397,PNT,3.0,--,0.2397,...,100.902124,0.2397,GALAXY,2.333333,3,0.0,0.2397,And a possible set of other lines that I coul...,3,disagreement in flag. Ha and OIII secure z. fl...


conflict ID =  44


In [205]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
348,35185766137073679,GALAXY,0.087,ACE,4.0,R,2.7396,GALAXY,Strong Lyman alpha and CIII] emission,2.7396,GALAXY,2.333333,3,0.087,2.440294,Strong Lyman alpha and CIII] emission| Low S/...,3,none
398,35185766137073679,GALAXY,0.087,TWL,1.0,--,--,--,Low S/N,0.0870,GALAXY,2.333333,3,0.087,2.440294,Strong Lyman alpha and CIII] emission| Low S/...,3,none
448,35185766137073679,GALAXY,0.087,PNT,2.0,S,0.0870,GALAXY,Noisy spectra OII doublet is not resolved Ha...,0.0870,GALAXY,2.333333,3,0.087,2.440294,Strong Lyman alpha and CIII] emission| Low S/...,3,none


In [206]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.7396
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
348,35185766137073679,-1,0.33.0.dev3890,0,GALAXY,0.087,ACE,2.0,R,2.7396,...,5.169612,2.7396,GALAXY,2.333333,3,2.440294,0.087,Strong Lyman alpha and CIII] emission| Low S/...,3,disagreement in z and flag. take VI z with fla...
398,35185766137073679,-1,0.33.0.dev3890,0,GALAXY,0.087,TWL,2.0,--,--,...,5.169612,2.7396,GALAXY,2.333333,3,2.440294,0.087,Strong Lyman alpha and CIII] emission| Low S/...,3,disagreement in z and flag. take VI z with fla...
448,35185766137073679,-1,0.33.0.dev3890,0,GALAXY,0.087,PNT,2.0,S,0.0870,...,5.169612,2.7396,GALAXY,2.333333,3,2.440294,0.087,Strong Lyman alpha and CIII] emission| Low S/...,3,disagreement in z and flag. take VI z with fla...


conflict ID =  45


In [209]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
450,35185766137073699,GALAXY,1.3973,CGS,1.0,--,--,--,--,1.3973,GALAXY,1.0,3,1.3973,0.0667,--|--|Low S/N|Based on OII in sky lines plus M...,4,none
500,35185766137073699,GALAXY,1.3973,BJW,0.0,--,--,--,--,1.3973,GALAXY,1.0,3,1.3973,0.0667,--|--|Low S/N|Based on OII in sky lines plus M...,4,none
550,35185766137073699,GALAXY,1.3973,TWL,0.0,--,--,--,Low S/N,1.3973,GALAXY,1.0,3,1.3973,0.0667,--|--|Low S/N|Based on OII in sky lines plus M...,4,none
600,35185766137073699,GALAXY,1.3973,ACE,3.0,R,1.5572,GALAXY,Based on OII in sky lines plus MgII and CIV in...,1.5572,GALAXY,1.0,3,1.3973,0.0667,--|--|Low S/N|Based on OII in sky lines plus M...,4,none


In [210]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3973
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. noisy spectra, take RR z with flag 1. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
450,35185766137073699,-1,0.33.0.dev3890,0,GALAXY,1.3973,CGS,1.0,--,--,...,2.305944,1.3973,GALAXY,1.0,3,0.0667,1.3973,--|--|Low S/N|Based on OII in sky lines plus M...,4,"disagreement in flag and z. noisy spectra, tak..."
500,35185766137073699,-1,0.33.0.dev3890,0,GALAXY,1.3973,BJW,1.0,--,--,...,2.305944,1.3973,GALAXY,1.0,3,0.0667,1.3973,--|--|Low S/N|Based on OII in sky lines plus M...,4,"disagreement in flag and z. noisy spectra, tak..."
550,35185766137073699,-1,0.33.0.dev3890,0,GALAXY,1.3973,TWL,1.0,--,--,...,2.305944,1.3973,GALAXY,1.0,3,0.0667,1.3973,--|--|Low S/N|Based on OII in sky lines plus M...,4,"disagreement in flag and z. noisy spectra, tak..."
600,35185766137073699,-1,0.33.0.dev3890,0,GALAXY,1.3973,ACE,1.0,R,1.5572,...,2.305944,1.3973,GALAXY,1.0,3,0.0667,1.3973,--|--|Low S/N|Based on OII in sky lines plus M...,4,"disagreement in flag and z. noisy spectra, tak..."


conflict ID =  46


In [213]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
456,35185766141265461,GALAXY,0.1849,CGS,1.0,--,--,--,--,0.1849,GALAXY,2.0,3,0.1849,0.0,--|--|--|--,4,none
506,35185766141265461,GALAXY,0.1849,BJW,2.0,--,--,--,--,0.1849,GALAXY,2.0,3,0.1849,0.0,--|--|--|--,4,none
556,35185766141265461,GALAXY,0.1849,TWL,1.0,--,--,--,--,0.1849,GALAXY,2.0,3,0.1849,0.0,--|--|--|--,4,none
606,35185766141265461,GALAXY,0.1849,ACE,4.0,--,--,--,--,0.1849,GALAXY,2.0,3,0.1849,0.0,--|--|--|--,4,none


In [214]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
456,35185766141265461,-1,0.33.0.dev3890,0,GALAXY,0.1849,CGS,2.0,--,--,...,21.878434,0.1849,GALAXY,2.0,3,0.0,0.1849,--|--|--|--,4,"flag disagreement, take mean."
506,35185766141265461,-1,0.33.0.dev3890,0,GALAXY,0.1849,BJW,2.0,--,--,...,21.878434,0.1849,GALAXY,2.0,3,0.0,0.1849,--|--|--|--,4,"flag disagreement, take mean."
556,35185766141265461,-1,0.33.0.dev3890,0,GALAXY,0.1849,TWL,2.0,--,--,...,21.878434,0.1849,GALAXY,2.0,3,0.0,0.1849,--|--|--|--,4,"flag disagreement, take mean."
606,35185766141265461,-1,0.33.0.dev3890,0,GALAXY,0.1849,ACE,2.0,--,--,...,21.878434,0.1849,GALAXY,2.0,3,0.0,0.1849,--|--|--|--,4,"flag disagreement, take mean."


conflict ID =  47


In [215]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
458,35185766141266056,GALAXY,1.4727,CGS,1.0,--,--,--,--,1.4727,GALAXY,1.0,3,1.4727,0.042828,--|--| Low S/N| OII in sky lines with CIV abso...,4,none
508,35185766141266056,GALAXY,1.4727,BJW,0.0,--,--,--,--,1.4727,GALAXY,1.0,3,1.4727,0.042828,--|--| Low S/N| OII in sky lines with CIV abso...,4,none
558,35185766141266056,GALAXY,1.4727,TWL,0.0,--,--,--,Low S/N,1.4727,GALAXY,1.0,3,1.4727,0.042828,--|--| Low S/N| OII in sky lines with CIV abso...,4,none
608,35185766141266056,GALAXY,1.4727,ACE,3.0,R,1.5786,GALAXY,OII in sky lines with CIV absorption?,1.5786,GALAXY,1.0,3,1.4727,0.042828,--|--| Low S/N| OII in sky lines with CIV abso...,4,none


In [216]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4727
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy spectra. RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
458,35185766141266056,-1,0.33.0.dev3890,0,GALAXY,1.4727,CGS,1.0,--,--,...,11.546023,1.4727,GALAXY,1.0,3,0.042828,1.4727,--|--| Low S/N| OII in sky lines with CIV abso...,4,disagreement in z and flag. noisy spectra. RR ...
508,35185766141266056,-1,0.33.0.dev3890,0,GALAXY,1.4727,BJW,1.0,--,--,...,11.546023,1.4727,GALAXY,1.0,3,0.042828,1.4727,--|--| Low S/N| OII in sky lines with CIV abso...,4,disagreement in z and flag. noisy spectra. RR ...
558,35185766141266056,-1,0.33.0.dev3890,0,GALAXY,1.4727,TWL,1.0,--,--,...,11.546023,1.4727,GALAXY,1.0,3,0.042828,1.4727,--|--| Low S/N| OII in sky lines with CIV abso...,4,disagreement in z and flag. noisy spectra. RR ...
608,35185766141266056,-1,0.33.0.dev3890,0,GALAXY,1.4727,ACE,1.0,R,1.5786,...,11.546023,1.4727,GALAXY,1.0,3,0.042828,1.4727,--|--| Low S/N| OII in sky lines with CIV abso...,4,disagreement in z and flag. noisy spectra. RR ...


conflict ID =  48


In [220]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
459,35185766141266205,GALAXY,1.5166,CGS,0.0,--,--,--,--,1.5166,GALAXY,1.25,3,1.5166,0.0,--| fitting sky residual|Possible [OII] emissi...,4,none
509,35185766141266205,GALAXY,1.5166,BJW,0.0,R,--,--,fitting sky residual,1.5166,GALAXY,1.25,3,1.5166,0.0,--| fitting sky residual|Possible [OII] emissi...,4,none
559,35185766141266205,GALAXY,1.5166,TWL,2.0,--,--,--,Possible [OII] emission next to sky lines; bei...,1.5166,GALAXY,1.25,3,1.5166,0.0,--| fitting sky residual|Possible [OII] emissi...,4,none
609,35185766141266205,GALAXY,1.5166,ACE,3.0,--,--,--,--,1.5166,GALAXY,1.25,3,1.5166,0.0,--| fitting sky residual|Possible [OII] emissi...,4,none


In [221]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.25
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
459,35185766141266205,-1,0.33.0.dev3890,0,GALAXY,1.5166,CGS,1.25,--,--,...,92.648552,1.5166,GALAXY,1.25,3,0.0,1.5166,--| fitting sky residual|Possible [OII] emissi...,4,"flag disagreement, take mean."
509,35185766141266205,-1,0.33.0.dev3890,0,GALAXY,1.5166,BJW,1.25,R,--,...,92.648552,1.5166,GALAXY,1.25,3,0.0,1.5166,--| fitting sky residual|Possible [OII] emissi...,4,"flag disagreement, take mean."
559,35185766141266205,-1,0.33.0.dev3890,0,GALAXY,1.5166,TWL,1.25,--,--,...,92.648552,1.5166,GALAXY,1.25,3,0.0,1.5166,--| fitting sky residual|Possible [OII] emissi...,4,"flag disagreement, take mean."
609,35185766141266205,-1,0.33.0.dev3890,0,GALAXY,1.5166,ACE,1.25,--,--,...,92.648552,1.5166,GALAXY,1.25,3,0.0,1.5166,--| fitting sky residual|Possible [OII] emissi...,4,"flag disagreement, take mean."


conflict ID =  49


In [224]:
#copy this text to a new cell to display the conflict
conflict_id=50
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
462,35185766141267125,GALAXY,1.0118,CGS,1.0,--,--,--,--,1.0118,GALAXY,2.25,2,1.011875,0.000149,--|--| Possible [OII] emission| Weak OII,4,none
512,35185766141267125,GALAXY,1.0118,BJW,3.0,--,--,--,--,1.0118,GALAXY,2.25,2,1.011875,0.000149,--|--| Possible [OII] emission| Weak OII,4,none
562,35185766141267125,GALAXY,1.0118,TWL,2.0,--,--,--,Possible [OII] emission,1.0118,GALAXY,2.25,2,1.011875,0.000149,--|--| Possible [OII] emission| Weak OII,4,none
612,35185766141267125,GALAXY,1.0118,ACE,3.0,--,1.0121,GALAXY,Weak OII,1.0121,GALAXY,2.25,2,1.011875,0.000149,--|--| Possible [OII] emission| Weak OII,4,none


In [226]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.25
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
462,35185766141267125,-1,0.33.0.dev3890,0,GALAXY,1.0118,CGS,2.25,--,--,...,3.994441,1.0118,GALAXY,2.25,2,0.000149,1.011875,--|--| Possible [OII] emission| Weak OII,4,"flag disagreement, take mean."
512,35185766141267125,-1,0.33.0.dev3890,0,GALAXY,1.0118,BJW,2.25,--,--,...,3.994441,1.0118,GALAXY,2.25,2,0.000149,1.011875,--|--| Possible [OII] emission| Weak OII,4,"flag disagreement, take mean."
562,35185766141267125,-1,0.33.0.dev3890,0,GALAXY,1.0118,TWL,2.25,--,--,...,3.994441,1.0118,GALAXY,2.25,2,0.000149,1.011875,--|--| Possible [OII] emission| Weak OII,4,"flag disagreement, take mean."
612,35185766141267125,-1,0.33.0.dev3890,0,GALAXY,1.0118,ACE,2.25,--,1.0121,...,3.994441,1.0121,GALAXY,2.25,2,0.000149,1.011875,--|--| Possible [OII] emission| Weak OII,4,"flag disagreement, take mean."


conflict ID =  50


In [229]:
#copy this text to a new cell to display the conflict
conflict_id=51
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
464,35185766141267656,GALAXY,0.7717,CGS,1.0,--,--,--,--,0.7717,GALAXY,1.25,3,0.7717,0.015185,--|--| Low S/N| Weak OII,4,none
514,35185766141267656,GALAXY,0.7717,BJW,1.0,--,--,--,--,0.7717,GALAXY,1.25,3,0.7717,0.015185,--|--| Low S/N| Weak OII,4,none
564,35185766141267656,GALAXY,0.7717,TWL,0.0,--,--,--,Low S/N,0.7717,GALAXY,1.25,3,0.7717,0.015185,--|--| Low S/N| Weak OII,4,none
614,35185766141267656,GALAXY,0.7717,ACE,3.0,R,0.7452,GALAXY,Weak OII,0.7452,GALAXY,1.25,3,0.7717,0.015185,--|--| Low S/N| Weak OII,4,none


In [230]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7717
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. noisy spectrum. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
464,35185766141267656,-1,0.33.0.dev3890,0,GALAXY,0.7717,CGS,1.0,--,--,...,1.782959,0.7717,GALAXY,1.25,3,0.015185,0.7717,--|--| Low S/N| Weak OII,4,disagreement in flag and z. noisy spectrum. ta...
514,35185766141267656,-1,0.33.0.dev3890,0,GALAXY,0.7717,BJW,1.0,--,--,...,1.782959,0.7717,GALAXY,1.25,3,0.015185,0.7717,--|--| Low S/N| Weak OII,4,disagreement in flag and z. noisy spectrum. ta...
564,35185766141267656,-1,0.33.0.dev3890,0,GALAXY,0.7717,TWL,1.0,--,--,...,1.782959,0.7717,GALAXY,1.25,3,0.015185,0.7717,--|--| Low S/N| Weak OII,4,disagreement in flag and z. noisy spectrum. ta...
614,35185766141267656,-1,0.33.0.dev3890,0,GALAXY,0.7717,ACE,1.0,R,0.7452,...,1.782959,0.7717,GALAXY,1.25,3,0.015185,0.7717,--|--| Low S/N| Weak OII,4,disagreement in flag and z. noisy spectrum. ta...


conflict ID =  51


In [231]:
#copy this text to a new cell to display the conflict
conflict_id=52
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
471,35185766141268985,GALAXY,1.0734,CGS,1.0,--,--,--,--,1.0734,GALAXY,1.0,3,1.0734,0.931619,--|--| Low S/N| From narrow Halpha and matchin...,4,none
521,35185766141268985,GALAXY,1.0734,BJW,0.0,R,--,--,--,1.0734,GALAXY,1.0,3,1.0734,0.931619,--|--| Low S/N| From narrow Halpha and matchin...,4,none
571,35185766141268985,GALAXY,1.0734,TWL,0.0,--,--,--,Low S/N,1.0734,GALAXY,1.0,3,1.0734,0.931619,--|--| Low S/N| From narrow Halpha and matchin...,4,none
621,35185766141268985,GALAXY,1.0734,ACE,3.0,R,0.0734,GALAXY,From narrow Halpha and matching weaker features,0.0734,GALAXY,1.0,3,1.0734,0.931619,--|--| Low S/N| From narrow Halpha and matchin...,4,none


In [232]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0734
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. noisy spectrum. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
471,35185766141268985,-1,0.33.0.dev3890,0,GALAXY,1.0734,CGS,1.0,--,--,...,3.624758,1.0734,GALAXY,1.0,3,0.931619,1.0734,--|--| Low S/N| From narrow Halpha and matchin...,4,disagreement in flag and z. noisy spectrum. ta...
521,35185766141268985,-1,0.33.0.dev3890,0,GALAXY,1.0734,BJW,1.0,R,--,...,3.624758,1.0734,GALAXY,1.0,3,0.931619,1.0734,--|--| Low S/N| From narrow Halpha and matchin...,4,disagreement in flag and z. noisy spectrum. ta...
571,35185766141268985,-1,0.33.0.dev3890,0,GALAXY,1.0734,TWL,1.0,--,--,...,3.624758,1.0734,GALAXY,1.0,3,0.931619,1.0734,--|--| Low S/N| From narrow Halpha and matchin...,4,disagreement in flag and z. noisy spectrum. ta...
621,35185766141268985,-1,0.33.0.dev3890,0,GALAXY,1.0734,ACE,1.0,R,0.0734,...,3.624758,1.0734,GALAXY,1.0,3,0.931619,1.0734,--|--| Low S/N| From narrow Halpha and matchin...,4,disagreement in flag and z. noisy spectrum. ta...


conflict ID =  52


In [235]:
#copy this text to a new cell to display the conflict
conflict_id=53
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
473,35185766141269081,GALAXY,0.9761,CGS,1.0,--,--,--,--,0.9761,GALAXY,1.0,3,0.9761,0.304236,--|--| Low S/N| OII in sky lines but clear,4,none
523,35185766141269081,GALAXY,0.9761,BJW,0.0,R,--,--,--,0.9761,GALAXY,1.0,3,0.9761,0.304236,--|--| Low S/N| OII in sky lines but clear,4,none
573,35185766141269081,GALAXY,0.9761,TWL,0.0,--,--,--,Low S/N,0.9761,GALAXY,1.0,3,0.9761,0.304236,--|--| Low S/N| OII in sky lines but clear,4,none
623,35185766141269081,GALAXY,0.9761,ACE,3.0,R,1.5773,GALAXY,OII in sky lines but clear,1.5773,GALAXY,1.0,3,0.9761,0.304236,--|--| Low S/N| OII in sky lines but clear,4,none


In [236]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5773
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. VI z based on weak OII in skyline. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
473,35185766141269081,-1,0.33.0.dev3890,0,GALAXY,0.9761,CGS,2.0,--,--,...,1.923174,1.5773,GALAXY,1.0,3,0.304236,0.9761,--|--| Low S/N| OII in sky lines but clear,4,disagreement in z and flag. VI z based on weak...
523,35185766141269081,-1,0.33.0.dev3890,0,GALAXY,0.9761,BJW,2.0,R,--,...,1.923174,1.5773,GALAXY,1.0,3,0.304236,0.9761,--|--| Low S/N| OII in sky lines but clear,4,disagreement in z and flag. VI z based on weak...
573,35185766141269081,-1,0.33.0.dev3890,0,GALAXY,0.9761,TWL,2.0,--,--,...,1.923174,1.5773,GALAXY,1.0,3,0.304236,0.9761,--|--| Low S/N| OII in sky lines but clear,4,disagreement in z and flag. VI z based on weak...
623,35185766141269081,-1,0.33.0.dev3890,0,GALAXY,0.9761,ACE,2.0,R,1.5773,...,1.923174,1.5773,GALAXY,1.0,3,0.304236,0.9761,--|--| Low S/N| OII in sky lines but clear,4,disagreement in z and flag. VI z based on weak...


conflict ID =  53


In [239]:
#copy this text to a new cell to display the conflict
conflict_id=54
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
474,35185766145458262,GALAXY,1.008,CGS,0.0,--,--,--,--,1.0080,GALAXY,1.0,3,1.008,0.08511,--|--|Some continuum no lines| Weak OII,4,none
524,35185766145458262,GALAXY,1.008,BJW,0.0,R,--,--,--,1.0080,GALAXY,1.0,3,1.008,0.08511,--|--|Some continuum no lines| Weak OII,4,none
574,35185766145458262,GALAXY,1.008,TWL,1.0,--,--,--,Some continuum no lines,1.0080,GALAXY,1.0,3,1.008,0.08511,--|--|Some continuum no lines| Weak OII,4,none
624,35185766145458262,GALAXY,1.008,ACE,3.0,R,1.1789,GALAXY,Weak OII,1.1789,GALAXY,1.0,3,1.008,0.08511,--|--|Some continuum no lines| Weak OII,4,none


In [240]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.008
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy spectrum. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
474,35185766145458262,-1,0.33.0.dev3890,0,GALAXY,1.008,CGS,1.0,--,--,...,7.625507,1.008,GALAXY,1.0,3,0.08511,1.008,--|--|Some continuum no lines| Weak OII,4,disagreement in z and flag. noisy spectrum. ta...
524,35185766145458262,-1,0.33.0.dev3890,0,GALAXY,1.008,BJW,1.0,R,--,...,7.625507,1.008,GALAXY,1.0,3,0.08511,1.008,--|--|Some continuum no lines| Weak OII,4,disagreement in z and flag. noisy spectrum. ta...
574,35185766145458262,-1,0.33.0.dev3890,0,GALAXY,1.008,TWL,1.0,--,--,...,7.625507,1.008,GALAXY,1.0,3,0.08511,1.008,--|--|Some continuum no lines| Weak OII,4,disagreement in z and flag. noisy spectrum. ta...
624,35185766145458262,-1,0.33.0.dev3890,0,GALAXY,1.008,ACE,1.0,R,1.1789,...,7.625507,1.008,GALAXY,1.0,3,0.08511,1.008,--|--|Some continuum no lines| Weak OII,4,disagreement in z and flag. noisy spectrum. ta...


conflict ID =  54


In [241]:
#copy this text to a new cell to display the conflict
conflict_id=55
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
481,35185766145459530,GALAXY,0.0315,CGS,1.0,--,--,--,--,0.0315,GALAXY,1.25,3,0.0315,2.129714,--|--| Low S/N| Based on CIV and NV,4,none
531,35185766145459530,GALAXY,0.0315,BJW,1.0,--,--,--,--,0.0315,GALAXY,1.25,3,0.0315,2.129714,--|--| Low S/N| Based on CIV and NV,4,none
581,35185766145459530,GALAXY,0.0315,TWL,0.0,--,--,--,Low S/N,0.0315,GALAXY,1.25,3,0.0315,2.129714,--|--| Low S/N| Based on CIV and NV,4,none
631,35185766145459530,GALAXY,0.0315,ACE,3.0,R,2.2283,GALAXY,Based on CIV and NV,2.2283,GALAXY,1.25,3,0.0315,2.129714,--|--| Low S/N| Based on CIV and NV,4,none


In [242]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0315
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy spectrum. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
481,35185766145459530,-1,0.33.0.dev3890,0,GALAXY,0.0315,CGS,1.0,--,--,...,3.056389,0.0315,GALAXY,1.25,3,2.129714,0.0315,--|--| Low S/N| Based on CIV and NV,4,disagreement in z and flag. noisy spectrum. ta...
531,35185766145459530,-1,0.33.0.dev3890,0,GALAXY,0.0315,BJW,1.0,--,--,...,3.056389,0.0315,GALAXY,1.25,3,2.129714,0.0315,--|--| Low S/N| Based on CIV and NV,4,disagreement in z and flag. noisy spectrum. ta...
581,35185766145459530,-1,0.33.0.dev3890,0,GALAXY,0.0315,TWL,1.0,--,--,...,3.056389,0.0315,GALAXY,1.25,3,2.129714,0.0315,--|--| Low S/N| Based on CIV and NV,4,disagreement in z and flag. noisy spectrum. ta...
631,35185766145459530,-1,0.33.0.dev3890,0,GALAXY,0.0315,ACE,1.0,R,2.2283,...,3.056389,0.0315,GALAXY,1.25,3,2.129714,0.0315,--|--| Low S/N| Based on CIV and NV,4,disagreement in z and flag. noisy spectrum. ta...


conflict ID =  55


In [245]:
#copy this text to a new cell to display the conflict
conflict_id=56
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
482,35185766145459538,GALAXY,0.9138,CGS,0.0,--,--,--,--,0.9138,GALAXY,1.5,3,0.9138,0.343244,--|--| Possible [OII] lines at 9590| Redrock c...,4,none
532,35185766145459538,GALAXY,0.9138,BJW,0.0,--,--,--,--,0.9138,GALAXY,1.5,3,0.9138,0.343244,--|--| Possible [OII] lines at 9590| Redrock c...,4,none
582,35185766145459538,GALAXY,0.9138,TWL,3.0,R,1.5707,--,Possible [OII] lines at 9590,1.5707,GALAXY,1.5,3,0.9138,0.343244,--|--| Possible [OII] lines at 9590| Redrock c...,4,none
632,35185766145459538,GALAXY,0.9138,ACE,3.0,R,1.5705,GALAXY,Redrock confuses OII for OIII,1.5705,GALAXY,1.5,3,0.9138,0.343244,--|--| Possible [OII] lines at 9590| Redrock c...,4,none


In [247]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5706
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/4 inspectors give VI z based on broadened but unresolved line. take answer with flag 2. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
482,35185766145459538,-1,0.33.0.dev3890,0,GALAXY,0.9138,CGS,2.0,--,--,...,2.005594,1.5706,GALAXY,1.5,3,0.343244,0.9138,--|--| Possible [OII] lines at 9590| Redrock c...,4,disagreement in z and flag. 2/4 inspectors giv...
532,35185766145459538,-1,0.33.0.dev3890,0,GALAXY,0.9138,BJW,2.0,--,--,...,2.005594,1.5706,GALAXY,1.5,3,0.343244,0.9138,--|--| Possible [OII] lines at 9590| Redrock c...,4,disagreement in z and flag. 2/4 inspectors giv...
582,35185766145459538,-1,0.33.0.dev3890,0,GALAXY,0.9138,TWL,2.0,R,1.5707,...,2.005594,1.5706,GALAXY,1.5,3,0.343244,0.9138,--|--| Possible [OII] lines at 9590| Redrock c...,4,disagreement in z and flag. 2/4 inspectors giv...
632,35185766145459538,-1,0.33.0.dev3890,0,GALAXY,0.9138,ACE,2.0,R,1.5705,...,2.005594,1.5706,GALAXY,1.5,3,0.343244,0.9138,--|--| Possible [OII] lines at 9590| Redrock c...,4,disagreement in z and flag. 2/4 inspectors giv...


conflict ID =  56


In [255]:
#copy this text to a new cell to display the conflict
conflict_id=57
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
484,35185766145459637,GALAXY,1.3931,CGS,1.75,--,--,--,--,1.3931,GALAXY,1.75,2,1.3931,0.0,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
534,35185766145459637,GALAXY,1.3931,BJW,1.75,--,--,--,--,1.3931,GALAXY,1.75,2,1.3931,0.0,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
584,35185766145459637,GALAXY,1.3931,TWL,1.75,--,--,--,Possible [OII] line next to sky lines,1.3931,GALAXY,1.75,2,1.3931,0.0,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
634,35185766145459637,GALAXY,1.3931,ACE,1.75,--,--,--,--,1.3931,GALAXY,1.75,2,1.3931,0.0,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...


In [258]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.75
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line on skylines. poor sky subtraction. take mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
484,35185766145459637,-1,0.33.0.dev3890,0,GALAXY,1.3931,CGS,1.75,--,--,...,34.074335,1.3931,GALAXY,1.75,2,0.0,1.3931,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
534,35185766145459637,-1,0.33.0.dev3890,0,GALAXY,1.3931,BJW,1.75,--,--,...,34.074335,1.3931,GALAXY,1.75,2,0.0,1.3931,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
584,35185766145459637,-1,0.33.0.dev3890,0,GALAXY,1.3931,TWL,1.75,--,--,...,34.074335,1.3931,GALAXY,1.75,2,0.0,1.3931,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...
634,35185766145459637,-1,0.33.0.dev3890,0,GALAXY,1.3931,ACE,1.75,--,--,...,34.074335,1.3931,GALAXY,1.75,2,0.0,1.3931,--|--|Possible [OII] line next to sky lines|--,4,disagreement in flag. broadened but unresolved...


conflict ID =  57


In [263]:
#copy this text to a new cell to display the conflict
conflict_id=58
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
486,35185766145459659,GALAXY,0.8978,CGS,4.0,--,--,--,--,0.8978,GALAXY,4.0,0,0.8978,0.0,--|--| Broad emission line| Broad OIII and NeV,4,none
536,35185766145459659,GALAXY,0.8978,BJW,4.0,--,--,--,--,0.8978,GALAXY,4.0,0,0.8978,0.0,--|--| Broad emission line| Broad OIII and NeV,4,none
586,35185766145459659,GALAXY,0.8978,TWL,4.0,--,--,--,Broad emission line,0.8978,GALAXY,4.0,0,0.8978,0.0,--|--| Broad emission line| Broad OIII and NeV,4,none
636,35185766145459659,GALAXY,0.8978,ACE,4.0,--,0.8978,QSO,Broad OIII and NeV,0.8978,QSO,4.0,0,0.8978,0.0,--|--| Broad emission line| Broad OIII and NeV,4,none


In [264]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. inspectors flag broad OIII and NeV. Broad NeIII seen but not on template. change to QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
486,35185766145459659,-1,0.33.0.dev3890,0,GALAXY,0.8978,CGS,4.0,--,--,...,1270.504351,0.8978,QSO,4.0,0,0.0,0.8978,--|--| Broad emission line| Broad OIII and NeV,4,disagreement in spectype. inspectors flag broa...
536,35185766145459659,-1,0.33.0.dev3890,0,GALAXY,0.8978,BJW,4.0,--,--,...,1270.504351,0.8978,QSO,4.0,0,0.0,0.8978,--|--| Broad emission line| Broad OIII and NeV,4,disagreement in spectype. inspectors flag broa...
586,35185766145459659,-1,0.33.0.dev3890,0,GALAXY,0.8978,TWL,4.0,--,--,...,1270.504351,0.8978,QSO,4.0,0,0.0,0.8978,--|--| Broad emission line| Broad OIII and NeV,4,disagreement in spectype. inspectors flag broa...
636,35185766145459659,-1,0.33.0.dev3890,0,GALAXY,0.8978,ACE,4.0,--,0.8978,...,1270.504351,0.8978,QSO,4.0,0,0.0,0.8978,--|--| Broad emission line| Broad OIII and NeV,4,disagreement in spectype. inspectors flag broa...


conflict ID =  58


In [267]:
#copy this text to a new cell to display the conflict
conflict_id=59
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
487,35185766145460293,GALAXY,0.9926,CGS,1.0,--,--,--,--,0.9926,GALAXY,1.5,2,0.9926,0.304226,--|--| Possible emission at 9690A|Redrock conf...,4,none
537,35185766145460293,GALAXY,0.9926,BJW,1.0,--,--,--,--,0.9926,GALAXY,1.5,2,0.9926,0.304226,--|--| Possible emission at 9690A|Redrock conf...,4,none
587,35185766145460293,GALAXY,0.9926,TWL,1.0,--,--,--,Possible emission at 9690A,0.9926,GALAXY,1.5,2,0.9926,0.304226,--|--| Possible emission at 9690A|Redrock conf...,4,none
637,35185766145460293,GALAXY,0.9926,ACE,3.0,R,1.5988,GALAXY,Redrock confuses OII for Hbeta,1.5988,GALAXY,1.5,2,0.9926,0.304226,--|--| Possible emission at 9690A|Redrock conf...,4,none


In [268]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5988
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. VI z based on broadened but unresolved line in skyline region. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
487,35185766145460293,-1,0.33.0.dev3890,0,GALAXY,0.9926,CGS,2.0,--,--,...,3.810536,1.5988,GALAXY,1.5,2,0.304226,0.9926,--|--| Possible emission at 9690A|Redrock conf...,4,disagreement in flag and z. VI z based on broa...
537,35185766145460293,-1,0.33.0.dev3890,0,GALAXY,0.9926,BJW,2.0,--,--,...,3.810536,1.5988,GALAXY,1.5,2,0.304226,0.9926,--|--| Possible emission at 9690A|Redrock conf...,4,disagreement in flag and z. VI z based on broa...
587,35185766145460293,-1,0.33.0.dev3890,0,GALAXY,0.9926,TWL,2.0,--,--,...,3.810536,1.5988,GALAXY,1.5,2,0.304226,0.9926,--|--| Possible emission at 9690A|Redrock conf...,4,disagreement in flag and z. VI z based on broa...
637,35185766145460293,-1,0.33.0.dev3890,0,GALAXY,0.9926,ACE,2.0,R,1.5988,...,3.810536,1.5988,GALAXY,1.5,2,0.304226,0.9926,--|--| Possible emission at 9690A|Redrock conf...,4,disagreement in flag and z. VI z based on broa...


conflict ID =  59


In [271]:
#copy this text to a new cell to display the conflict
conflict_id=60
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
488,35185766145460342,GALAXY,0.1413,CGS,4.0,--,--,--,--,0.1413,GALAXY,3.5,2,0.1413,0.0,--|--| Not sure the fitting result is signific...,4,none
538,35185766145460342,GALAXY,0.1413,BJW,4.0,--,--,--,--,0.1413,GALAXY,3.5,2,0.1413,0.0,--|--| Not sure the fitting result is signific...,4,none
588,35185766145460342,GALAXY,0.1413,TWL,2.0,--,--,--,Not sure the fitting result is significant,0.1413,GALAXY,3.5,2,0.1413,0.0,--|--| Not sure the fitting result is signific...,4,none
638,35185766145460342,GALAXY,0.1413,ACE,4.0,--,--,--,--,0.1413,GALAXY,3.5,2,0.1413,0.0,--|--| Not sure the fitting result is signific...,4,none


In [272]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
488,35185766145460342,-1,0.33.0.dev3890,0,GALAXY,0.1413,CGS,3.5,--,--,...,79.66318,0.1413,GALAXY,3.5,2,0.0,0.1413,--|--| Not sure the fitting result is signific...,4,"flag disagreement, take mean."
538,35185766145460342,-1,0.33.0.dev3890,0,GALAXY,0.1413,BJW,3.5,--,--,...,79.66318,0.1413,GALAXY,3.5,2,0.0,0.1413,--|--| Not sure the fitting result is signific...,4,"flag disagreement, take mean."
588,35185766145460342,-1,0.33.0.dev3890,0,GALAXY,0.1413,TWL,3.5,--,--,...,79.66318,0.1413,GALAXY,3.5,2,0.0,0.1413,--|--| Not sure the fitting result is signific...,4,"flag disagreement, take mean."
638,35185766145460342,-1,0.33.0.dev3890,0,GALAXY,0.1413,ACE,3.5,--,--,...,79.66318,0.1413,GALAXY,3.5,2,0.0,0.1413,--|--| Not sure the fitting result is signific...,4,"flag disagreement, take mean."


conflict ID =  60


In [275]:
#copy this text to a new cell to display the conflict
conflict_id=61
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
489,35185766145460344,GALAXY,1.137,CGS,4.0,--,--,--,--,1.137,GALAXY,3.0,2,1.137,0.0,--|--|Possible [OII] next to sky lines|--,4,none
539,35185766145460344,GALAXY,1.137,BJW,3.0,--,--,--,--,1.137,GALAXY,3.0,2,1.137,0.0,--|--|Possible [OII] next to sky lines|--,4,none
589,35185766145460344,GALAXY,1.137,TWL,2.0,--,--,--,Possible [OII] next to sky lines,1.137,GALAXY,3.0,2,1.137,0.0,--|--|Possible [OII] next to sky lines|--,4,none
639,35185766145460344,GALAXY,1.137,ACE,3.0,--,--,--,--,1.137,GALAXY,3.0,2,1.137,0.0,--|--|Possible [OII] next to sky lines|--,4,none


In [276]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. continuum suggests OII. set to mean flag of 3'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
489,35185766145460344,-1,0.33.0.dev3890,0,GALAXY,1.137,CGS,3.0,--,--,...,17.600712,1.137,GALAXY,3.0,2,0.0,1.137,--|--|Possible [OII] next to sky lines|--,4,disagreement in flag. broadened but unresolved...
539,35185766145460344,-1,0.33.0.dev3890,0,GALAXY,1.137,BJW,3.0,--,--,...,17.600712,1.137,GALAXY,3.0,2,0.0,1.137,--|--|Possible [OII] next to sky lines|--,4,disagreement in flag. broadened but unresolved...
589,35185766145460344,-1,0.33.0.dev3890,0,GALAXY,1.137,TWL,3.0,--,--,...,17.600712,1.137,GALAXY,3.0,2,0.0,1.137,--|--|Possible [OII] next to sky lines|--,4,disagreement in flag. broadened but unresolved...
639,35185766145460344,-1,0.33.0.dev3890,0,GALAXY,1.137,ACE,3.0,--,--,...,17.600712,1.137,GALAXY,3.0,2,0.0,1.137,--|--|Possible [OII] next to sky lines|--,4,disagreement in flag. broadened but unresolved...


conflict ID =  61


In [279]:
#copy this text to a new cell to display the conflict
conflict_id=62
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
687,35185766145462140,GALAXY,1.2066,ACE,3.0,R,0.9345,GALAXY,Based on weak OII and Hdelta.,0.9345,GALAXY,0.75,3,1.2066,0.140657,Based on weak OII and Hdelta.|no signal| Low ...,4,none
737,35185766145462140,GALAXY,1.2066,BJW,0.0,R,--,--,no signal,1.2066,GALAXY,0.75,3,1.2066,0.140657,Based on weak OII and Hdelta.|no signal| Low ...,4,none
787,35185766145462140,GALAXY,1.2066,TWL,0.0,--,--,--,Low S/N,1.2066,GALAXY,0.75,3,1.2066,0.140657,Based on weak OII and Hdelta.|no signal| Low ...,4,none
837,35185766145462140,GALAXY,1.2066,CGS,0.0,--,--,--,--,1.2066,GALAXY,0.75,3,1.2066,0.140657,Based on weak OII and Hdelta.|no signal| Low ...,4,none


In [280]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2066
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy spectrum, no secure z. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
687,35185766145462140,-1,0.33.0.dev3890,0,GALAXY,1.2066,ACE,1.0,R,0.9345,...,7.131087,1.2066,GALAXY,0.75,3,0.140657,1.2066,Based on weak OII and Hdelta.|no signal| Low ...,4,"disagreement in z and flag. noisy spectrum, no..."
737,35185766145462140,-1,0.33.0.dev3890,0,GALAXY,1.2066,BJW,1.0,R,--,...,7.131087,1.2066,GALAXY,0.75,3,0.140657,1.2066,Based on weak OII and Hdelta.|no signal| Low ...,4,"disagreement in z and flag. noisy spectrum, no..."
787,35185766145462140,-1,0.33.0.dev3890,0,GALAXY,1.2066,TWL,1.0,--,--,...,7.131087,1.2066,GALAXY,0.75,3,0.140657,1.2066,Based on weak OII and Hdelta.|no signal| Low ...,4,"disagreement in z and flag. noisy spectrum, no..."
837,35185766145462140,-1,0.33.0.dev3890,0,GALAXY,1.2066,CGS,1.0,--,--,...,7.131087,1.2066,GALAXY,0.75,3,0.140657,1.2066,Based on weak OII and Hdelta.|no signal| Low ...,4,"disagreement in z and flag. noisy spectrum, no..."


conflict ID =  62


In [284]:
#copy this text to a new cell to display the conflict
conflict_id=63
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
689,35185766145462649,GALAXY,0.9188,ACE,4.0,R,1.5775,GALAXY,Redrock confuses OII for OIII. MgII in emission.,1.5775,GALAXY,2.0,4,0.9188,0.343287,Redrock confuses OII for OIII. MgII in emissi...,4,none
739,35185766145462649,GALAXY,0.9188,BJW,0.0,R,--,--,--,0.9188,GALAXY,2.0,4,0.9188,0.343287,Redrock confuses OII for OIII. MgII in emissi...,4,none
789,35185766145462649,GALAXY,0.9188,TWL,2.0,--,1.5774,--,Possible [OII] at 9610 A next to sky lines,1.5774,GALAXY,2.0,4,0.9188,0.343287,Redrock confuses OII for OIII. MgII in emissi...,4,none
839,35185766145462649,GALAXY,0.9188,CGS,2.0,R,1.5775,--,--,1.5775,GALAXY,2.0,4,0.9188,0.343287,Redrock confuses OII for OIII. MgII in emissi...,4,none


In [285]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5774
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 8/3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 3/4 inspectors suggest VI z based on OII in skyline region. take z with their mean flag of 2.7'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
689,35185766145462649,-1,0.33.0.dev3890,0,GALAXY,0.9188,ACE,2.666667,R,1.5775,...,14.636959,1.5774,GALAXY,2.0,4,0.343287,0.9188,Redrock confuses OII for OIII. MgII in emissi...,4,disagreement in flag and z. 3/4 inspectors sug...
739,35185766145462649,-1,0.33.0.dev3890,0,GALAXY,0.9188,BJW,2.666667,R,--,...,14.636959,1.5774,GALAXY,2.0,4,0.343287,0.9188,Redrock confuses OII for OIII. MgII in emissi...,4,disagreement in flag and z. 3/4 inspectors sug...
789,35185766145462649,-1,0.33.0.dev3890,0,GALAXY,0.9188,TWL,2.666667,--,1.5774,...,14.636959,1.5774,GALAXY,2.0,4,0.343287,0.9188,Redrock confuses OII for OIII. MgII in emissi...,4,disagreement in flag and z. 3/4 inspectors sug...
839,35185766145462649,-1,0.33.0.dev3890,0,GALAXY,0.9188,CGS,2.666667,R,1.5775,...,14.636959,1.5774,GALAXY,2.0,4,0.343287,0.9188,Redrock confuses OII for OIII. MgII in emissi...,4,disagreement in flag and z. 3/4 inspectors sug...


conflict ID =  63


In [288]:
#copy this text to a new cell to display the conflict
conflict_id=64
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
690,35185766145462677,GALAXY,1.4972,ACE,3.0,R,0.2551,GALAXY,"Based on OII"","" NeII and Hdelta.",0.2551,GALAXY,1.0,3,1.4972,0.989642,"Based on OII"","" NeII and Hdelta.|sky residual...",4,none
740,35185766145462677,GALAXY,1.4972,BJW,0.0,R,--,--,sky residual,1.4972,GALAXY,1.0,3,1.4972,0.989642,"Based on OII"","" NeII and Hdelta.|sky residual...",4,none
790,35185766145462677,GALAXY,1.4972,TWL,0.0,--,--,--,Model fits sky lines,1.4972,GALAXY,1.0,3,1.4972,0.989642,"Based on OII"","" NeII and Hdelta.|sky residual...",4,none
840,35185766145462677,GALAXY,1.4972,CGS,1.0,--,--,--,--,1.4972,GALAXY,1.0,3,1.4972,0.989642,"Based on OII"","" NeII and Hdelta.|sky residual...",4,none


In [289]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4972
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. noisy spectrum, no secure z. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
690,35185766145462677,-1,0.33.0.dev3890,0,GALAXY,1.4972,ACE,1.0,R,0.2551,...,20.225732,1.4972,GALAXY,1.0,3,0.989642,1.4972,"Based on OII"","" NeII and Hdelta.|sky residual...",4,"disagreement in flag and z. noisy spectrum, no..."
740,35185766145462677,-1,0.33.0.dev3890,0,GALAXY,1.4972,BJW,1.0,R,--,...,20.225732,1.4972,GALAXY,1.0,3,0.989642,1.4972,"Based on OII"","" NeII and Hdelta.|sky residual...",4,"disagreement in flag and z. noisy spectrum, no..."
790,35185766145462677,-1,0.33.0.dev3890,0,GALAXY,1.4972,TWL,1.0,--,--,...,20.225732,1.4972,GALAXY,1.0,3,0.989642,1.4972,"Based on OII"","" NeII and Hdelta.|sky residual...",4,"disagreement in flag and z. noisy spectrum, no..."
840,35185766145462677,-1,0.33.0.dev3890,0,GALAXY,1.4972,CGS,1.0,--,--,...,20.225732,1.4972,GALAXY,1.0,3,0.989642,1.4972,"Based on OII"","" NeII and Hdelta.|sky residual...",4,"disagreement in flag and z. noisy spectrum, no..."


conflict ID =  64


In [290]:
#copy this text to a new cell to display the conflict
conflict_id=65
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
692,35185766145462814,GALAXY,1.4796,ACE,3.0,R,1.4716,GALAXY,Based on very weak OII and red stellar contin...,1.4716,GALAXY,1.0,3,1.4776,0.003237,Based on very weak OII and red stellar contin...,4,none
742,35185766145462814,GALAXY,1.4796,BJW,0.0,R,--,--,--,1.4796,GALAXY,1.0,3,1.4776,0.003237,Based on very weak OII and red stellar contin...,4,none
792,35185766145462814,GALAXY,1.4796,TWL,1.0,--,--,--,--,1.4796,GALAXY,1.0,3,1.4776,0.003237,Based on very weak OII and red stellar contin...,4,none
842,35185766145462814,GALAXY,1.4796,CGS,0.0,--,--,--,--,1.4796,GALAXY,1.0,3,1.4776,0.003237,Based on very weak OII and red stellar contin...,4,none


In [291]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4796
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. noisy spectrum, no secure z. take RR z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
692,35185766145462814,-1,0.33.0.dev3890,0,GALAXY,1.4796,ACE,1.0,R,1.4716,...,10.423264,1.4796,GALAXY,1.0,3,0.003237,1.4776,Based on very weak OII and red stellar contin...,4,"disagreement in flag and z. noisy spectrum, no..."
742,35185766145462814,-1,0.33.0.dev3890,0,GALAXY,1.4796,BJW,1.0,R,--,...,10.423264,1.4796,GALAXY,1.0,3,0.003237,1.4776,Based on very weak OII and red stellar contin...,4,"disagreement in flag and z. noisy spectrum, no..."
792,35185766145462814,-1,0.33.0.dev3890,0,GALAXY,1.4796,TWL,1.0,--,--,...,10.423264,1.4796,GALAXY,1.0,3,0.003237,1.4776,Based on very weak OII and red stellar contin...,4,"disagreement in flag and z. noisy spectrum, no..."
842,35185766145462814,-1,0.33.0.dev3890,0,GALAXY,1.4796,CGS,1.0,--,--,...,10.423264,1.4796,GALAXY,1.0,3,0.003237,1.4776,Based on very weak OII and red stellar contin...,4,"disagreement in flag and z. noisy spectrum, no..."


conflict ID =  65


In [294]:
#copy this text to a new cell to display the conflict
conflict_id=66
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
650,35185772134924401,GALAXY,0.444,ACE,4.0,RS,0.9898,GALAXY,Poor continuum subtraction 5800-7600A Redrock ...,0.9898,GALAXY,2.5,4,0.444,0.377978,Poor continuum subtraction 5800-7600A Redrock ...,4,none
700,35185772134924401,GALAXY,0.444,BJW,0.0,RS,--,--,Bad continuum in middle,0.4440,GALAXY,2.5,4,0.444,0.377978,Poor continuum subtraction 5800-7600A Redrock ...,4,none
750,35185772134924401,GALAXY,0.444,TWL,3.0,--,0.9895,--,Calibration features around 5800 A; possible [...,0.9895,GALAXY,2.5,4,0.444,0.377978,Poor continuum subtraction 5800-7600A Redrock ...,4,none
800,35185772134924401,GALAXY,0.444,CGS,3.0,RCS,0.9896,--,--,0.9896,GALAXY,2.5,4,0.444,0.377978,Poor continuum subtraction 5800-7600A Redrock ...,4,none


In [295]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9896
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/4 inspectors suggest same VI z, secured by doublet. calibration issues. flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
650,35185772134924401,-1,0.33.0.dev3890,0,GALAXY,0.444,ACE,3.0,RS,0.9898,...,6.559626,0.9896,GALAXY,2.5,4,0.377978,0.444,Poor continuum subtraction 5800-7600A Redrock ...,4,"3/4 inspectors suggest same VI z, secured by d..."
700,35185772134924401,-1,0.33.0.dev3890,0,GALAXY,0.444,BJW,3.0,RS,--,...,6.559626,0.9896,GALAXY,2.5,4,0.377978,0.444,Poor continuum subtraction 5800-7600A Redrock ...,4,"3/4 inspectors suggest same VI z, secured by d..."
750,35185772134924401,-1,0.33.0.dev3890,0,GALAXY,0.444,TWL,3.0,--,0.9895,...,6.559626,0.9896,GALAXY,2.5,4,0.377978,0.444,Poor continuum subtraction 5800-7600A Redrock ...,4,"3/4 inspectors suggest same VI z, secured by d..."
800,35185772134924401,-1,0.33.0.dev3890,0,GALAXY,0.444,CGS,3.0,RCS,0.9896,...,6.559626,0.9896,GALAXY,2.5,4,0.377978,0.444,Poor continuum subtraction 5800-7600A Redrock ...,4,"3/4 inspectors suggest same VI z, secured by d..."


conflict ID =  66


In [298]:
#copy this text to a new cell to display the conflict
conflict_id=67
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
652,35185772139118651,GALAXY,0.4996,ACE,4.0,RS,0.9019,GALAXY,Poor continuum subtraction 5800-7600A. Redroc...,0.9019,GALAXY,2.75,4,0.4996,0.268405,Poor continuum subtraction 5800-7600A. Redroc...,4,none
702,35185772139118651,GALAXY,0.4996,BJW,0.0,RS,--,--,Bad continuum,0.4996,GALAXY,2.75,4,0.4996,0.268405,Poor continuum subtraction 5800-7600A. Redroc...,4,none
752,35185772139118651,GALAXY,0.4996,TWL,3.0,S,0.9020,--,Calibration features around 5800 A; possible ...,0.9020,GALAXY,2.75,4,0.4996,0.268405,Poor continuum subtraction 5800-7600A. Redroc...,4,none
802,35185772139118651,GALAXY,0.4996,CGS,4.0,RCS,0.9021,--,--,0.9021,GALAXY,2.75,4,0.4996,0.268405,Poor continuum subtraction 5800-7600A. Redroc...,4,none


In [299]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9020
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/4 inspectors suggest same VI z based on resolved OII. flag 3. calibration issues'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
652,35185772139118651,-1,0.33.0.dev3890,0,GALAXY,0.4996,ACE,3.0,RS,0.9019,...,412.622289,0.902,GALAXY,2.75,4,0.268405,0.4996,Poor continuum subtraction 5800-7600A. Redroc...,4,3/4 inspectors suggest same VI z based on reso...
702,35185772139118651,-1,0.33.0.dev3890,0,GALAXY,0.4996,BJW,3.0,RS,--,...,412.622289,0.902,GALAXY,2.75,4,0.268405,0.4996,Poor continuum subtraction 5800-7600A. Redroc...,4,3/4 inspectors suggest same VI z based on reso...
752,35185772139118651,-1,0.33.0.dev3890,0,GALAXY,0.4996,TWL,3.0,S,0.9020,...,412.622289,0.902,GALAXY,2.75,4,0.268405,0.4996,Poor continuum subtraction 5800-7600A. Redroc...,4,3/4 inspectors suggest same VI z based on reso...
802,35185772139118651,-1,0.33.0.dev3890,0,GALAXY,0.4996,CGS,3.0,RCS,0.9021,...,412.622289,0.902,GALAXY,2.75,4,0.268405,0.4996,Poor continuum subtraction 5800-7600A. Redroc...,4,3/4 inspectors suggest same VI z based on reso...


conflict ID =  67


In [302]:
#copy this text to a new cell to display the conflict
conflict_id=68
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
653,35185772139118887,GALAXY,0.4344,ACE,4.0,RS,1.4185,GALAXY,Poor continuum subtraction 5800-7600A.,1.4185,GALAXY,1.75,4,0.4344,0.686071,Poor continuum subtraction 5800-7600A.| Bad c...,4,none
703,35185772139118887,GALAXY,0.4344,BJW,0.0,S,--,--,Bad continuum,0.4344,GALAXY,1.75,4,0.4344,0.686071,Poor continuum subtraction 5800-7600A.| Bad c...,4,none
753,35185772139118887,GALAXY,0.4344,TWL,2.0,S,--,--,Calibration features around 5800 A; possible ...,0.4344,GALAXY,1.75,4,0.4344,0.686071,Poor continuum subtraction 5800-7600A.| Bad c...,4,none
803,35185772139118887,GALAXY,0.4344,CGS,1.0,CS,--,--,--,0.4344,GALAXY,1.75,4,0.4344,0.686071,Poor continuum subtraction 5800-7600A.| Bad c...,4,none


In [303]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4344
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] =1 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. too faint for secure z. take RR z with flag 1. poor sky subtraction. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
653,35185772139118887,-1,0.33.0.dev3890,0,GALAXY,0.4344,ACE,1.0,RS,1.4185,...,13.443407,0.4344,GALAXY,1.75,4,0.686071,0.4344,Poor continuum subtraction 5800-7600A.| Bad c...,4,disagreement in z and flag. too faint for secu...
703,35185772139118887,-1,0.33.0.dev3890,0,GALAXY,0.4344,BJW,1.0,S,--,...,13.443407,0.4344,GALAXY,1.75,4,0.686071,0.4344,Poor continuum subtraction 5800-7600A.| Bad c...,4,disagreement in z and flag. too faint for secu...
753,35185772139118887,-1,0.33.0.dev3890,0,GALAXY,0.4344,TWL,1.0,S,--,...,13.443407,0.4344,GALAXY,1.75,4,0.686071,0.4344,Poor continuum subtraction 5800-7600A.| Bad c...,4,disagreement in z and flag. too faint for secu...
803,35185772139118887,-1,0.33.0.dev3890,0,GALAXY,0.4344,CGS,1.0,CS,--,...,13.443407,0.4344,GALAXY,1.75,4,0.686071,0.4344,Poor continuum subtraction 5800-7600A.| Bad c...,4,disagreement in z and flag. too faint for secu...


conflict ID =  68


In [306]:
#copy this text to a new cell to display the conflict
conflict_id=69
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
654,35185772139118906,GALAXY,0.6794,ACE,4.0,R,1.1005,GALAXY,"Continuum """"hump"""" 6300-7600A. Redrock misses...",1.1005,GALAXY,2.0,3,0.6794,0.250744,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,none
704,35185772139118906,GALAXY,0.6794,BJW,1.0,S,--,--,Bad continuum,0.6794,GALAXY,2.0,3,0.6794,0.250744,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,none
754,35185772139118906,GALAXY,0.6794,TWL,2.0,S,1.1000,--,Calibration features; possible [OII] emission...,1.1000,GALAXY,2.0,3,0.6794,0.250744,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,none
804,35185772139118906,GALAXY,0.6794,CGS,1.0,CS,--,--,--,0.6794,GALAXY,2.0,3,0.6794,0.250744,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,none


In [307]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1000
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/4 inspectors suggest VI z based on weak broadened OII on sky region. take answer with flag 2. calibration issues. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
654,35185772139118906,-1,0.33.0.dev3890,0,GALAXY,0.6794,ACE,2.0,R,1.1005,...,17.131973,1.1,GALAXY,2.0,3,0.250744,0.6794,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,disagreement in flag and z. 2/4 inspectors sug...
704,35185772139118906,-1,0.33.0.dev3890,0,GALAXY,0.6794,BJW,2.0,S,--,...,17.131973,1.1,GALAXY,2.0,3,0.250744,0.6794,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,disagreement in flag and z. 2/4 inspectors sug...
754,35185772139118906,-1,0.33.0.dev3890,0,GALAXY,0.6794,TWL,2.0,S,1.1000,...,17.131973,1.1,GALAXY,2.0,3,0.250744,0.6794,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,disagreement in flag and z. 2/4 inspectors sug...
804,35185772139118906,-1,0.33.0.dev3890,0,GALAXY,0.6794,CGS,2.0,CS,--,...,17.131973,1.1,GALAXY,2.0,3,0.250744,0.6794,"Continuum """"hump"""" 6300-7600A. Redrock misses...",4,disagreement in flag and z. 2/4 inspectors sug...


conflict ID =  69


In [310]:
#copy this text to a new cell to display the conflict
conflict_id=70
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
657,35185772143313215,GALAXY,0.6864,ACE,4.0,RS,0.7453,GALAXY,Very poor continuum subtract in blue and red ...,0.7453,GALAXY,2.25,4,0.6864,0.034926,Very poor continuum subtract in blue and red ...,4,none
707,35185772143313215,GALAXY,0.6864,BJW,0.0,RS,--,--,Bad continuum,0.6864,GALAXY,2.25,4,0.6864,0.034926,Very poor continuum subtract in blue and red ...,4,none
757,35185772143313215,GALAXY,0.6864,TWL,1.0,S,--,--,Calibration features,0.6864,GALAXY,2.25,4,0.6864,0.034926,Very poor continuum subtract in blue and red ...,4,none
807,35185772143313215,GALAXY,0.6864,CGS,4.0,RS,0.7452,--,--,0.7452,GALAXY,2.25,4,0.6864,0.034926,Very poor continuum subtract in blue and red ...,4,none


In [311]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7452
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/4 inspectors suggest same VI z, take with flag 3. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
657,35185772143313215,-1,0.33.0.dev3890,0,GALAXY,0.6864,ACE,2.0,RS,0.7453,...,14.127967,0.7452,GALAXY,2.25,4,0.034926,0.6864,Very poor continuum subtract in blue and red ...,4,disagreement in z and flag. 2/4 inspectors sug...
707,35185772143313215,-1,0.33.0.dev3890,0,GALAXY,0.6864,BJW,2.0,RS,--,...,14.127967,0.7452,GALAXY,2.25,4,0.034926,0.6864,Very poor continuum subtract in blue and red ...,4,disagreement in z and flag. 2/4 inspectors sug...
757,35185772143313215,-1,0.33.0.dev3890,0,GALAXY,0.6864,TWL,2.0,S,--,...,14.127967,0.7452,GALAXY,2.25,4,0.034926,0.6864,Very poor continuum subtract in blue and red ...,4,disagreement in z and flag. 2/4 inspectors sug...
807,35185772143313215,-1,0.33.0.dev3890,0,GALAXY,0.6864,CGS,2.0,RS,0.7452,...,14.127967,0.7452,GALAXY,2.25,4,0.034926,0.6864,Very poor continuum subtract in blue and red ...,4,disagreement in z and flag. 2/4 inspectors sug...


conflict ID =  70


In [314]:
#copy this text to a new cell to display the conflict
conflict_id= 71
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
658,35185772143315112,GALAXY,0.4472,ACE,3.0,RS,1.2192,GALAXY,Poor continuum subtraction 5800-7600A. Redroc...,1.2192,GALAXY,1.75,3,0.4472,0.533444,Poor continuum subtraction 5800-7600A. Redroc...,4,none
708,35185772143315112,GALAXY,0.4472,BJW,0.0,RS,--,--,Bad continuum,0.4472,GALAXY,1.75,3,0.4472,0.533444,Poor continuum subtraction 5800-7600A. Redroc...,4,none
758,35185772143315112,GALAXY,0.4472,TWL,1.0,S,--,--,Calibration features around 5800A,0.4472,GALAXY,1.75,3,0.4472,0.533444,Poor continuum subtraction 5800-7600A. Redroc...,4,none
808,35185772143315112,GALAXY,0.4472,CGS,3.0,RS,1.2191,--,--,1.2191,GALAXY,1.75,3,0.4472,0.533444,Poor continuum subtraction 5800-7600A. Redroc...,4,none


In [315]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2191
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/4 inspectors give VI z based on missed doublet. secure z. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
658,35185772143315112,-1,0.33.0.dev3890,0,GALAXY,0.4472,ACE,3.0,RS,1.2192,...,82.821314,1.2191,GALAXY,1.75,3,0.533444,0.4472,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
708,35185772143315112,-1,0.33.0.dev3890,0,GALAXY,0.4472,BJW,3.0,RS,--,...,82.821314,1.2191,GALAXY,1.75,3,0.533444,0.4472,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
758,35185772143315112,-1,0.33.0.dev3890,0,GALAXY,0.4472,TWL,3.0,S,--,...,82.821314,1.2191,GALAXY,1.75,3,0.533444,0.4472,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
808,35185772143315112,-1,0.33.0.dev3890,0,GALAXY,0.4472,CGS,3.0,RS,1.2191,...,82.821314,1.2191,GALAXY,1.75,3,0.533444,0.4472,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...


conflict ID =  71


In [318]:
#copy this text to a new cell to display the conflict
conflict_id=72
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
659,35185772147507555,GALAXY,0.6547,ACE,3.0,R,1.5173,GALAXY,OII in sky lines not clear but sufficient for...,1.5173,GALAXY,1.25,3,0.6547,0.521303,OII in sky lines not clear but sufficient for...,4,none
709,35185772147507555,GALAXY,0.6547,BJW,0.0,RS,--,--,Bad continuum,0.6547,GALAXY,1.25,3,0.6547,0.521303,OII in sky lines not clear but sufficient for...,4,none
759,35185772147507555,GALAXY,0.6547,TWL,1.0,S,--,--,Calibration features around 5800 A,0.6547,GALAXY,1.25,3,0.6547,0.521303,OII in sky lines not clear but sufficient for...,4,none
809,35185772147507555,GALAXY,0.6547,CGS,1.0,--,--,--,--,0.6547,GALAXY,1.25,3,0.6547,0.521303,OII in sky lines not clear but sufficient for...,4,none


In [319]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6547
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure z. take RR z with flag 1. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
659,35185772147507555,-1,0.33.0.dev3890,0,GALAXY,0.6547,ACE,1.0,R,1.5173,...,9.039033,0.6547,GALAXY,1.25,3,0.521303,0.6547,OII in sky lines not clear but sufficient for...,4,disagreement in flag and z. no secure z. take ...
709,35185772147507555,-1,0.33.0.dev3890,0,GALAXY,0.6547,BJW,1.0,RS,--,...,9.039033,0.6547,GALAXY,1.25,3,0.521303,0.6547,OII in sky lines not clear but sufficient for...,4,disagreement in flag and z. no secure z. take ...
759,35185772147507555,-1,0.33.0.dev3890,0,GALAXY,0.6547,TWL,1.0,S,--,...,9.039033,0.6547,GALAXY,1.25,3,0.521303,0.6547,OII in sky lines not clear but sufficient for...,4,disagreement in flag and z. no secure z. take ...
809,35185772147507555,-1,0.33.0.dev3890,0,GALAXY,0.6547,CGS,1.0,--,--,...,9.039033,0.6547,GALAXY,1.25,3,0.521303,0.6547,OII in sky lines not clear but sufficient for...,4,disagreement in flag and z. no secure z. take ...


conflict ID =  72


In [322]:
#copy this text to a new cell to display the conflict
conflict_id=73
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
661,35185772147508676,GALAXY,0.4467,ACE,3.0,RS,1.5913,GALAXY,Poor continuum subtraction 5800-7600A. OII in ...,1.5913,GALAXY,1.0,3,0.4467,0.79118,Poor continuum subtraction 5800-7600A. OII in ...,4,none
711,35185772147508676,GALAXY,0.4467,BJW,0.0,RS,--,--,Bad continuum,0.4467,GALAXY,1.0,3,0.4467,0.79118,Poor continuum subtraction 5800-7600A. OII in ...,4,none
761,35185772147508676,GALAXY,0.4467,TWL,0.0,S,--,--,Calibration features around 5800 A,0.4467,GALAXY,1.0,3,0.4467,0.79118,Poor continuum subtraction 5800-7600A. OII in ...,4,none
811,35185772147508676,GALAXY,0.4467,CGS,1.0,S,--,--,--,0.4467,GALAXY,1.0,3,0.4467,0.79118,Poor continuum subtraction 5800-7600A. OII in ...,4,none


In [323]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4467
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. z not secure. take RR answer with flag 1. calibration issues. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
661,35185772147508676,-1,0.33.0.dev3890,0,GALAXY,0.4467,ACE,1.0,RS,1.5913,...,11.44825,0.4467,GALAXY,1.0,3,0.79118,0.4467,Poor continuum subtraction 5800-7600A. OII in ...,4,disagreement in z and flag. z not secure. take...
711,35185772147508676,-1,0.33.0.dev3890,0,GALAXY,0.4467,BJW,1.0,RS,--,...,11.44825,0.4467,GALAXY,1.0,3,0.79118,0.4467,Poor continuum subtraction 5800-7600A. OII in ...,4,disagreement in z and flag. z not secure. take...
761,35185772147508676,-1,0.33.0.dev3890,0,GALAXY,0.4467,TWL,1.0,S,--,...,11.44825,0.4467,GALAXY,1.0,3,0.79118,0.4467,Poor continuum subtraction 5800-7600A. OII in ...,4,disagreement in z and flag. z not secure. take...
811,35185772147508676,-1,0.33.0.dev3890,0,GALAXY,0.4467,CGS,1.0,S,--,...,11.44825,0.4467,GALAXY,1.0,3,0.79118,0.4467,Poor continuum subtraction 5800-7600A. OII in ...,4,disagreement in z and flag. z not secure. take...


conflict ID =  73


In [327]:
#copy this text to a new cell to display the conflict
conflict_id=74
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
662,35185772147508976,GALAXY,0.4462,ACE,4.0,RS,0.9665,GALAXY,"Poor continuum subtraction 5800-7600A"","" Redr...",0.9665,GALAXY,2.75,2,0.966375,0.000153,"Poor continuum subtraction 5800-7600A"","" Redr...",4,none
712,35185772147508976,GALAXY,0.4462,BJW,2.0,RS,0.9663,GALAXY,Bad continuum - probable [O II] line,0.9663,GALAXY,2.75,2,0.966375,0.000153,"Poor continuum subtraction 5800-7600A"","" Redr...",4,none
762,35185772147508976,GALAXY,0.4462,TWL,3.0,--,0.9662,--,Calibration features around 5800 A; possible ...,0.9662,GALAXY,2.75,2,0.966375,0.000153,"Poor continuum subtraction 5800-7600A"","" Redr...",4,none
812,35185772147508976,GALAXY,0.4462,CGS,2.0,RS,0.9665,--,--,0.9665,GALAXY,2.75,2,0.966375,0.000153,"Poor continuum subtraction 5800-7600A"","" Redr...",4,none


In [328]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9663
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.75
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but 4/4 of inspectors give same VI z. Take with mean flag. Calibration issues. Poor sky subtraction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
662,35185772147508976,-1,0.33.0.dev3890,0,GALAXY,0.4462,ACE,2.75,RS,0.9665,...,16.727113,0.9663,GALAXY,2.75,2,0.000153,0.966375,"Poor continuum subtraction 5800-7600A"","" Redr...",4,"disagreement in flag, but 4/4 of inspectors gi..."
712,35185772147508976,-1,0.33.0.dev3890,0,GALAXY,0.4462,BJW,2.75,RS,0.9663,...,16.727113,0.9663,GALAXY,2.75,2,0.000153,0.966375,"Poor continuum subtraction 5800-7600A"","" Redr...",4,"disagreement in flag, but 4/4 of inspectors gi..."
762,35185772147508976,-1,0.33.0.dev3890,0,GALAXY,0.4462,TWL,2.75,--,0.9662,...,16.727113,0.9663,GALAXY,2.75,2,0.000153,0.966375,"Poor continuum subtraction 5800-7600A"","" Redr...",4,"disagreement in flag, but 4/4 of inspectors gi..."
812,35185772147508976,-1,0.33.0.dev3890,0,GALAXY,0.4462,CGS,2.75,RS,0.9665,...,16.727113,0.9663,GALAXY,2.75,2,0.000153,0.966375,"Poor continuum subtraction 5800-7600A"","" Redr...",4,"disagreement in flag, but 4/4 of inspectors gi..."


conflict ID =  74


In [331]:
#copy this text to a new cell to display the conflict
conflict_id=75
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
666,35185772147509695,GALAXY,0.4433,ACE,3.0,RS,1.2351,GALAXY,Poor continuum subtraction 5800-7600A. Redroc...,1.2351,GALAXY,1.5,3,0.4433,0.548604,Poor continuum subtraction 5800-7600A. Redroc...,4,none
716,35185772147509695,GALAXY,0.4433,BJW,0.0,RS,--,--,Bad continuum,0.4433,GALAXY,1.5,3,0.4433,0.548604,Poor continuum subtraction 5800-7600A. Redroc...,4,none
766,35185772147509695,GALAXY,0.4433,TWL,2.0,S,1.2349,--,Possible [OII] emission at 8340; calibration ...,1.2349,GALAXY,1.5,3,0.4433,0.548604,Poor continuum subtraction 5800-7600A. Redroc...,4,none
816,35185772147509695,GALAXY,0.4433,CGS,1.0,S,--,--,--,0.4433,GALAXY,1.5,3,0.4433,0.548604,Poor continuum subtraction 5800-7600A. Redroc...,4,none


In [332]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2350
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/4 inspectors give same VI based on broadened but unresolved line. z not secure. poor sky subtraction. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
666,35185772147509695,-1,0.33.0.dev3890,0,GALAXY,0.4433,ACE,2.0,RS,1.2351,...,85.717942,1.235,GALAXY,1.5,3,0.548604,0.4433,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in flag and z. 2/4 inspectors giv...
716,35185772147509695,-1,0.33.0.dev3890,0,GALAXY,0.4433,BJW,2.0,RS,--,...,85.717942,1.235,GALAXY,1.5,3,0.548604,0.4433,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in flag and z. 2/4 inspectors giv...
766,35185772147509695,-1,0.33.0.dev3890,0,GALAXY,0.4433,TWL,2.0,S,1.2349,...,85.717942,1.235,GALAXY,1.5,3,0.548604,0.4433,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in flag and z. 2/4 inspectors giv...
816,35185772147509695,-1,0.33.0.dev3890,0,GALAXY,0.4433,CGS,2.0,S,--,...,85.717942,1.235,GALAXY,1.5,3,0.548604,0.4433,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in flag and z. 2/4 inspectors giv...


conflict ID =  75


In [335]:
#copy this text to a new cell to display the conflict
conflict_id=76
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
668,35185772147510282,GALAXY,0.4537,ACE,4.0,RS,1.2784,GALAXY,Poor continuum subtraction 5800-7600A. Redrok...,1.2784,GALAXY,2.25,3,0.4537,0.56738,Poor continuum subtraction 5800-7600A. Redrok...,4,none
718,35185772147510282,GALAXY,0.4537,BJW,2.0,RS,1.2785,GALAXY,Bad continuum. possible [O II] line,1.2785,GALAXY,2.25,3,0.4537,0.56738,Poor continuum subtraction 5800-7600A. Redrok...,4,none
768,35185772147510282,GALAXY,0.4537,TWL,2.0,S,1.2783,--,Possible [OII] emission at 8500; Calibration ...,1.2783,GALAXY,2.25,3,0.4537,0.56738,Poor continuum subtraction 5800-7600A. Redrok...,4,none
818,35185772147510282,GALAXY,0.4537,CGS,1.0,S,--,--,--,0.4537,GALAXY,2.25,3,0.4537,0.56738,Poor continuum subtraction 5800-7600A. Redrok...,4,none


In [336]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2785
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 3/4 inspectors give VI z based on broadened but unresolved line on sky line. z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
668,35185772147510282,-1,0.33.0.dev3890,0,GALAXY,0.4537,ACE,2.0,RS,1.2784,...,36.373775,1.2785,GALAXY,2.25,3,0.56738,0.4537,Poor continuum subtraction 5800-7600A. Redrok...,4,disagreement in flag and z. 3/4 inspectors giv...
718,35185772147510282,-1,0.33.0.dev3890,0,GALAXY,0.4537,BJW,2.0,RS,1.2785,...,36.373775,1.2785,GALAXY,2.25,3,0.56738,0.4537,Poor continuum subtraction 5800-7600A. Redrok...,4,disagreement in flag and z. 3/4 inspectors giv...
768,35185772147510282,-1,0.33.0.dev3890,0,GALAXY,0.4537,TWL,2.0,S,1.2783,...,36.373775,1.2785,GALAXY,2.25,3,0.56738,0.4537,Poor continuum subtraction 5800-7600A. Redrok...,4,disagreement in flag and z. 3/4 inspectors giv...
818,35185772147510282,-1,0.33.0.dev3890,0,GALAXY,0.4537,CGS,2.0,S,--,...,36.373775,1.2785,GALAXY,2.25,3,0.56738,0.4537,Poor continuum subtraction 5800-7600A. Redrok...,4,disagreement in flag and z. 3/4 inspectors giv...


conflict ID =  76


In [339]:
#copy this text to a new cell to display the conflict
conflict_id=77
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
672,35185772147511014,GALAXY,0.4462,ACE,4.0,RS,1.0152,GALAXY,Poor continuum subtraction 5800-7600A. Redroc...,1.0152,GALAXY,1.75,4,0.4462,0.393514,Poor continuum subtraction 5800-7600A. Redroc...,4,none
722,35185772147511014,GALAXY,0.4462,BJW,0.0,RS,--,--,Bad continuum,0.4462,GALAXY,1.75,4,0.4462,0.393514,Poor continuum subtraction 5800-7600A. Redroc...,4,none
772,35185772147511014,GALAXY,0.4462,TWL,0.0,S,--,--,Calibration features around 5800 A,0.4462,GALAXY,1.75,4,0.4462,0.393514,Poor continuum subtraction 5800-7600A. Redroc...,4,none
822,35185772147511014,GALAXY,0.4462,CGS,3.0,RS,1.0153,--,--,1.0153,GALAXY,1.75,4,0.4462,0.393514,Poor continuum subtraction 5800-7600A. Redroc...,4,none


In [340]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4462
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/4 inspectors give same VI z based on weak broadened but unresolved line. z not secure. poor sky subtraction. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
672,35185772147511014,-1,0.33.0.dev3890,0,GALAXY,0.4462,ACE,2.0,RS,1.0152,...,0.596926,0.4462,GALAXY,1.75,4,0.393514,0.4462,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
722,35185772147511014,-1,0.33.0.dev3890,0,GALAXY,0.4462,BJW,2.0,RS,--,...,0.596926,0.4462,GALAXY,1.75,4,0.393514,0.4462,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
772,35185772147511014,-1,0.33.0.dev3890,0,GALAXY,0.4462,TWL,2.0,S,--,...,0.596926,0.4462,GALAXY,1.75,4,0.393514,0.4462,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...
822,35185772147511014,-1,0.33.0.dev3890,0,GALAXY,0.4462,CGS,2.0,RS,1.0153,...,0.596926,0.4462,GALAXY,1.75,4,0.393514,0.4462,Poor continuum subtraction 5800-7600A. Redroc...,4,disagreement in z and flag. 2/4 inspectors giv...


conflict ID =  77


In [341]:
#copy this text to a new cell to display the conflict
conflict_id= 78
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
674,35185772147511266,GALAXY,0.4516,ACE,4.0,RS,1.4169,GALAXY,Poor continuum subtraction 5800-7600A. MgII in...,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,none
724,35185772147511266,GALAXY,0.4516,BJW,0.0,RS,--,--,Bad continuum,0.4516,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,none
774,35185772147511266,GALAXY,0.4516,TWL,3.0,RS,1.4169,--,Calibration features around 5800 A; [OII] emi...,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,none
824,35185772147511266,GALAXY,0.4516,CGS,3.0,RS,1.4169,--,--,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,none


In [342]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, z secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
674,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,ACE,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
724,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,BJW,3.0,RS,--,...,5.92965,0.4516,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
774,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,TWL,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
824,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,CGS,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."


conflict ID =  78


In [345]:
#copy this text to a new cell to display the conflict
conflict_id=79
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
683,35185772147511802,GALAXY,1.2627,ACE,4.0,--,--,--,--,1.2627,GALAXY,3.25,2,1.2627,0.0,--|--|Possible [OII] lines next to sky lines; ...,4,none
733,35185772147511802,GALAXY,1.2627,BJW,3.0,--,--,--,--,1.2627,GALAXY,3.25,2,1.2627,0.0,--|--|Possible [OII] lines next to sky lines; ...,4,none
783,35185772147511802,GALAXY,1.2627,TWL,2.0,--,--,--,Possible [OII] lines next to sky lines; being ...,1.2627,GALAXY,3.25,2,1.2627,0.0,--|--|Possible [OII] lines next to sky lines; ...,4,none
833,35185772147511802,GALAXY,1.2627,CGS,4.0,--,--,--,--,1.2627,GALAXY,3.25,2,1.2627,0.0,--|--|Possible [OII] lines next to sky lines; ...,4,none


In [346]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
683,35185772147511802,-1,0.33.0.dev3890,0,GALAXY,1.2627,ACE,2.0,--,--,...,196.763429,1.2627,GALAXY,3.25,2,0.0,1.2627,--|--|Possible [OII] lines next to sky lines; ...,4,disagreement in flag. broadened but unresolved...
733,35185772147511802,-1,0.33.0.dev3890,0,GALAXY,1.2627,BJW,2.0,--,--,...,196.763429,1.2627,GALAXY,3.25,2,0.0,1.2627,--|--|Possible [OII] lines next to sky lines; ...,4,disagreement in flag. broadened but unresolved...
783,35185772147511802,-1,0.33.0.dev3890,0,GALAXY,1.2627,TWL,2.0,--,--,...,196.763429,1.2627,GALAXY,3.25,2,0.0,1.2627,--|--|Possible [OII] lines next to sky lines; ...,4,disagreement in flag. broadened but unresolved...
833,35185772147511802,-1,0.33.0.dev3890,0,GALAXY,1.2627,CGS,2.0,--,--,...,196.763429,1.2627,GALAXY,3.25,2,0.0,1.2627,--|--|Possible [OII] lines next to sky lines; ...,4,disagreement in flag. broadened but unresolved...


conflict ID =  79


### and so on...

We should now recompute the conflicts, and not find any.

In [349]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [350]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185772147511266]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [351]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
674,35185772147511266,GALAXY,0.4516,ACE,3.0,RS,1.4169,GALAXY,Poor continuum subtraction 5800-7600A. MgII in...,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
724,35185772147511266,GALAXY,0.4516,BJW,3.0,RS,--,--,Bad continuum,0.4516,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
774,35185772147511266,GALAXY,0.4516,TWL,3.0,RS,1.4169,--,Calibration features around 5800 A; [OII] emi...,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
824,35185772147511266,GALAXY,0.4516,CGS,3.0,RS,1.4169,--,--,1.4169,GALAXY,2.5,4,0.4516,0.66499,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."


In [353]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4169
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
674,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,ACE,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
724,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,BJW,3.0,RS,--,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
774,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,TWL,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."
824,35185772147511266,-1,0.33.0.dev3890,0,GALAXY,0.4516,CGS,3.0,RS,1.4169,...,5.92965,1.4169,GALAXY,2.5,4,0.66499,0.4516,Poor continuum subtraction 5800-7600A. MgII in...,4,"flag disagreement, z secure."


conflict ID =  0


Let's check the number of conflicts again.

In [354]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [355]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [356]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [357]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [269]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [270]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [271]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'DELTACHI2', 'FIBER',
       'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z'],
      dtype='object')

In [272]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35191325322120560,0.1012,GALAXY,0.1012,GALAXY,4.000000,13,2248.126427,500,8.915575,13.690263,18.567017,3.849107,5.910476,8.015910
1,35191325326313329,0.2555,GALAXY,0.2555,GALAXY,4.000000,13,597.103415,471,3.061106,6.622557,11.148005,1.005650,2.175676,3.662399
2,35191325326313441,0.1513,GALAXY,0.1513,GALAXY,3.923077,13,402.013473,459,8.835087,15.250219,22.150007,1.184792,2.045066,2.970333
3,35191325326313842,0.1255,GALAXY,0.1255,GALAXY,4.000000,12,1171.531194,356,7.692234,12.353596,16.146137,2.066740,3.319149,4.338124
4,35191325326313983,0.0213,GALAXY,0.0213,GALAXY,4.000000,13,1233.152205,398,9.700712,11.533047,10.710887,1.310380,1.557893,1.446835
5,35191325326314275,0.5843,GALAXY,0.5843,GALAXY,3.769231,13,215.043012,191,3.716881,10.441963,23.580444,0.818709,2.300028,5.194012
6,35191325326314402,0.4220,GALAXY,0.4220,GALAXY,4.000000,13,1241.922597,397,4.036667,10.297829,18.527788,1.408013,3.591943,6.462602
7,35191325326314538,0.4593,GALAXY,0.4593,GALAXY,3.000000,13,360.383502,363,1.326949,6.981154,19.884447,0.448803,2.361180,6.725359
8,35191325326314565,0.2785,GALAXY,0.2785,GALAXY,3.769231,13,487.159330,192,3.047705,12.858685,27.577822,1.625618,6.858705,14.709757
9,35191325326314860,0.3363,GALAXY,0.3363,GALAXY,3.000000,13,284.628837,184,2.184791,9.017674,19.233717,1.025388,4.232265,9.026961


In [ ]:
merged_file.loc[9]['merger comment']

In [ ]:
merged_file.loc[9]['all VI comments']

In [273]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/BGS/'